In [1]:
import pandas as pd #18/02/2023! Diane
import numpy as np
import pdfplumber
import shutil
import glob, os, re

path_transporeon = r"C:/Users/john.tan/Documents/Transporeon/"
path_wms = r"C:/Users/john.tan/Documents/WMS/"
dir_remov = [path_transporeon+"Json/",
             path_transporeon+"Transporeon_split/", 
             path_transporeon+"Transporeon_split/Transporeon_split_1/",
             path_transporeon+"Chawan_Vopak/", 
             path_transporeon+"Send_Vopak/"]
for dir_remo in dir_remov:
    for file_remov in glob.glob(dir_remo +"*.pdf"):
        os.remove(file_remov)

from pathlib import Path #remove file from dir
dir_path = Path('C:/Users/john.tan/Documents/Transporeon/')
pdf_files = dir_path.glob('*.pdf') # returns a list, get all pdf files in directory
# check if specific name exists in list
for pdf_file in pdf_files:
    if 'Transports_Number_.pdf' in pdf_file.name:
        os.remove(pdf_file)

#move file, overwrite if exists
list_of_files = glob.glob('C:/Users/john.tan/Downloads/*.pdf') # *.csv
latest_file = max(list_of_files, key=os.path.getctime) #get latest files
src_path = r"C:/Users/john.tan/Downloads/"
dest_path = path_transporeon

def substring_after(s, delim):
    return s.partition(delim)[2]

latest_transport = substring_after(latest_file, "\\")
if os.path.isfile(os.path.join(dest_path, latest_transport)):
    os.remove(os.path.join(dest_path, latest_transport)) #remove file if exists
shutil.move(src_path+latest_transport, dest_path)


from PyPDF2 import PdfWriter, PdfReader #split pdf files
from PyPDF2 import PdfWriter, PdfReader
path_file = path_transporeon + latest_transport
file_out = path_transporeon + "Transporeon_split/Transport"
inputpdf = PdfReader(open(path_file, "rb"))

for i in range(len(inputpdf.pages)):
    output = PdfWriter()
    output.add_page(inputpdf.pages[i])
    with open(file_out + "_%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)
        
substr = 'Transporeon-ID' #remove pdf w/o 'Transporeon-ID'
path_dir = path_transporeon + "Transporeon_split/"
for file in glob.glob(path_dir+"*.pdf"):
    file = str(file)
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    
    if text.find(substr) !=-1:
        path_from = file
        #print(file)
        path_dest = path_transporeon + 'Transporeon_split_1/'
        shutil.move(path_from, path_dest)
        pdf.close()

        
regexp = re.compile("Origin(.*)$", re.MULTILINE)
path_dir_1 = path_transporeon + "Transporeon_split_1/"
path_in = path_transporeon + "Chawan_Vopak/" #move pdf to 'Chawan', 'Vopak'
for file in glob.glob(path_dir_1 +"*.pdf"):
    file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori in ['CHAWAN', 'VOPAK']:
            shutil.move(path_from, path_in)
            pdf.close()
    except:
        pass

In [2]:
dir = path_transporeon + r"Chawan_Vopak/" #regex rename pdf Diane
regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
regexp_2 = re.compile(r'(?s)Shipping number (BOL) (.*?)\n[^\S\r\n]*Gross ', re.MULTILINE)
regexp_3 = re.compile(r'Shipping number (BOL) (\d+)', re.MULTILINE)
regexp_4 = re.compile(r'(?s)Order No. (.*?)\n[^\S\r\n]*Order ')

ad=0
for file in os.listdir(dir):
    f = os.path.join(dir, file) # iterate over files in dir
    if os.path.isfile(f): # checking if it is a file
        pdf = pdfplumber.open(f)
        page = pdf.pages[0]
        text = page.extract_text()
        pdf.close()
        
        cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', text) #MV3
        
        dic_cust={'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant MITSUI ELASTOMERS SINGAPORE PTE LTD 1 Ayer Chawan Place  701 AYER MERBAU ROAD SG SG Singapore': 'MITSUI ELASTOMERS SINGAPORE PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal BASF SOUTH EAST ASIA PTE LTD PENJURU ROAD 59 20 BANYAN AVENUE JURONG ISLAND SG SG Singapore': 'BASF SOUTH EAST ASIA PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal THE POLYOLEFIN -PULAU AYER MERBAU PENJURU ROAD 59 Ayer Merbau Road 401 SG SG Singapore': 'THE POLYOLEFIN -PULAU AYER MERBAU', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal MEGACHEM MANUFACTURING PTE LTD PENJURU ROAD 59 PIONEER ROAD 132 SG SG-639588 SINGAPORE': 'MEGACHEM MANUFACTURING PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal CHAMPIONX SG SERVICE PTE LTD PENJURU ROAD 59 11 AYER MERBAU ROAD, JURONG ISLAND SG SG-627526 SINGAPORE': 'CHAMPIONX SG SERVICE PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal CLP INTERNATIONAL PTE LTD PENJURU ROAD 59 2 TANJONG PENJURU SG SG-609017 SINGAPORE': 'CLP INTERNATIONAL PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal EXXONMOBIL CHEMICAL COMPANY PENJURU ROAD 59 JURONG ISLAND HIGHWAY 100 SG SG-627867 SINGAPORE': 'EXXONMOBIL CHEMICAL COMPANY', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal A.I.M. CHEMICAL INDUSTRIES PTE LTD PENJURU ROAD 59 19 TRACTOR ROAD SG SG-627977 SINGAPORE (DDP)': 'A.I.M. CHEMICAL INDUSTRIES PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal SUMITOMO CHEMICAL ASIA PTE LTD PENJURU ROAD 59 AYER MERBAU ROAD 700 SG SG-627525 SINGAPORE': 'SUMITOMO CHEMICAL ASIA PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD PENJURU ROAD 59 1 ANGSANA DRIVE JURONG ISLAND SG SG Singapore': 'ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal CLP INTERNATIONAL PTE LTD PENJURU ROAD 59 2 TANJONG PENJURU SG SG-609017 SINGAPORE': 'CLP INTERNATIONAL PTE LTD', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal MSD INTERNATIONAL GMBH (SINGAPORE BRANCH) PENJURU ROAD 59 50 TUAS WEST DRIVE SG SG-638408 SINGAPORE': 'MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal SINGAPORE ADHESION PLANT PENJURU ROAD 59 100 JURONG ISLAND HIGHWAY SG SG Singapore': 'SINGAPORE ADHESION PLANT', 
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG PENJURU ROAD 59 TEL : 68676725 / 65342276 SG SG Singapore': 'SUMITOMO SEIKA-17', 
          'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant SUMITOMO CHEMICAL ASIA PTE LTD 1 Ayer Chawan Place  AYER MERBAU ROAD 700 SG SG-627525 SINGAPORE': 'SUMITOMO CHEMICAL ASIA PTE LTD', 
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal MITSUI ELASTOMERS SINGAPORE PTE LTD PENJURU ROAD 59 701 AYER MERBAU ROAD SG SG Singapore': 'MITSUI ELASTOMERS SINGAPORE PTE LTD',
          'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal TOTALENERGIES MARKETING ASIA-PACIFIC MIDDLE EAST PTE LTD PENJURU ROAD 59 (LOBP) SG TUAS SOUTH AVENUE 14 2': 'TOTALENERGIES MARKETING ASIAPACIFIC',
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal BERTSCHI SINGAPORE PTE. LTD. PENJURU ROAD 59 SERAYA PLACE, JURONG ISLAND 2 SG SG Singapore': 'BERTSCHI SINGAPORE PTE. LTD.',
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal THE POLYOLEFIN - PULAU AYER MERBAU PENJURU ROAD 59 STREET 2 GATE NO 5 401 SG SG Singapore': 'THE POLYOLEFIN - PULAU AYER MERBAU G5',
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal THE POLYOLEFIN - PULAU AYER MERBAU PENJURU ROAD 59 STREET 2 GATE NO 23 401 SG SG Singapore': 'THE POLYOLEFIN - PULAU AYER MERBAU G23',
          'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant MITSUI ELASTOMERS SINGAPORE PTE LTD 1 Ayer Chawan Place 701 AYER MERBAU ROAD SG SG Singapore': 'MITSUI ELASTOMERS SINGAPORE PTE LTD',
          'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal MEGACHEM MANUFACTURING PTE LTD PENJURU ROAD 59 132 PIONEER ROAD SG SG-639588 SINGAPORE': 'MEGACHEM MANUFACTURING PTE LTD',
           'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant CLP INTERNATIONAL PTE LTD 1 Ayer Chawan Place 2 TANJONG PENJURU SG SG-609017 SINGAPORE': 'CLP INTERNATIONAL PTE LTD',
           'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal MSD INTERNATIONAL GMBH (SINGAPORE BRANCH) PENJURU ROAD 59 50 TUAS WEST DRIVE SG SG-638408 SINGAPORE': 'MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)',
           'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal SINGAPORE BUTYL PLANT PENJURU ROAD 59 100 JURONG ISLAND HIGHWAY SG SG Singapore': 'SINGAPORE BUTYL PLANT',
           'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant SUMITOMO CHEMICAL ASIA PTE LTD 1 Ayer Chawan Place AYER MERBAU ROAD 700 SG SG-627525 SINGAPORE': 'SUMITOMO CHEMICAL ASIA PTE LTD',
           'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant MITSUI ELASTOMERS SINGAPORE PTE LTD 1 AYER CHAWAN PLACE 701 AYER MERBAU ROAD SG SG Singapore': 'MITSUI ELASTOMERS SINGAPORE PTE LTD',
           'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal THE POLYOLEFIN - PULAU AYER MERBAU PENJURU ROAD 59 STREET 2 GATE NO 23 401 SG SG Singapore': 'THE POLYOLEFIN_G23',
           'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant MITSUI ELASTOMERS SINGAPORE PTE LTD 1 AYER CHAWAN PLACE 701 AYER MERBAU ROAD SG SG Singapore': 'MITSUI ELASTOMERS SINGAPORE',
           'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant CLP INTERNATIONAL PTE LTD 1 AYER CHAWAN PLACE 2 TANJONG PENJURU SG SG-609017 SINGAPORE': 'CLP INTERNATIONAL PTE LTD',
           'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant SUMITOMO CHEMICAL ASIA PTE LTD 1 AYER CHAWAN PLACE AYER MERBAU ROAD 700 SG SG-627525 SINGAPORE': 'SUMITOMO CHEMICAL ASIA PL',
           'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal A.I.M. CHEMICAL INDUSTRIES PTE LTD PENJURU ROAD 59 19 TRACTOR ROAD SG SG-627977 SINGAPORE (DDP)': 'AIM CHEMICAL INDUSTRIES',
           'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal SINGAPORE ADHESION PLANT PENJURU ROAD 59 100 JURONG ISLAND HIGHWAY SG SG Singapore': 'SINGAPORE ADHESION PLANT',
           'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal SH COGENT LOGISTICS JURONG ISLAND LOGISTICS HUB PENJURU ROAD 59 TEMBUSU CRESCENT 15 SG SG Singapore': 'SHCOGENT',
           'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal CLP INTERNATIONAL PTE LTD PENJURU ROAD 59 2 TANJONG PENJURU SG SG-609017 SINGAPORE': 'CLP INTERNATIPNAL PTE LTD'
           }
        
        custy = " ".join(line.strip() for line in cust[0].splitlines())
        cust_renam = dic_cust.get(custy)
        
        try:
            if re.search("BASF SOUTH EAST ASIA PTE LTD", cust_renam):
                try:
                    r = r"SAP\sOrder\sNo..*?(?=\w)(\w+)"
                    ship_num = re.findall(r, text)[0]
                    f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +'.pdf'
                    os.rename(f, f1)
                except:
                    pass
        except:
            pass                                
        try:
            ship_num = regexp.search(text).group(1) #
            ship_num = ".".join(re.findall(r'\d+', ship_num))
            f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass
        try:
            ship_num = regexp_2.search(text).group(1) #
            f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +'.pdf'
            os.rename(f, f1)
        except:
            pass
        if cust_renam == 'MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)':
            try:
                r = r"SAP\sOrder\sNo..*?(?=\w)(\w+)"
                ship_num = re.findall(r, text)[0]
                f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +'.pdf'
                os.rename(f, f1)
            except:
                pass
        if cust_renam == 'SINGAPORE ADHESION PLANT':
            text =text.replace('\n', '')
            try:
                r = r"SGSAP\sOrder\sNo. *?(?=\w)(\w+)"
                ship_num = re.findall(r, text)[0]
                ship_num = re.findall('[0-9]+', ship_num)[0]
                f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +'.pdf'
                os.rename(f, f1)
                ad +=1        
            except:
                    pass
            #ad +=1
            #f1 = f.rpartition('/')[0] + '/'+ cust_renam + '_' + ship_num +"_"+ str(ad) + '.pdf'
            #os.rename(f, f1)

In [ ]:
from azure.core.credentials import AzureKeyCredential #optional
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.serialization import AzureJSONEncoder
#from azure.ai.formrecognizer import AnalyzeResult
#import json

endpoint = "https://...cognitiveservices.azure.com/"
key = ""

path_dir = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Chawan_Vopak/"
path_destination = "C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Json/"
i = 1

document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key))

list_of_files = glob.glob(path_dir+'*.pdf')
for file in list_of_files:
    file = str(file)
    path_to_sample_documents = file
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-document", document=f
        )
    result = poller.result()

    analyze_result_dict = result.to_dict()
    with open(path_destination + 'data' + "_" +str(i)+ ".json", 'w') as f:
        json.dump(analyze_result_dict, f, cls=AzureJSONEncoder)
    i += 1


In [4]:
from azure.core.credentials import AzureKeyCredential #Recognizer key_value pairs 18/2/2023! Diane
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.serialization import AzureJSONEncoder
import json

endpoint = "https://for-transporean.cognitiveservices.azure.com/"
key = ""

path_pdf = path_transporeon+"Chawan_Vopak/"
path_json = path_transporeon+"Json/"

i=0
lst_trans = []
lst_dfq = []
field_lst = ['Shipping Number (BOL)', 'Comment', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']#MV2
field_lst2 = ['Shipping Number (BOL)', 'comment:', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']
field_lst5 = ['Shipping Number (BOL)', 'Transport comment:', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']
field_lst3 = ['Shipping Number (BOL)', 'Comment', 'Transport No.', 'SAP Order No.', 'Article description']
field_lst4 = ['Shipping Number (BOL)', 'Transport No.', 'SAP Order No.', 'Origin'] #basf
field_lst5 = ['Shipping Number (BOL)', 'Transport comment:', 'Transport No.', 'Origin', 'SAP Order No.', 'Article description']
field_lst6 = ['Shipping Number (BOL)', 'Transport No.', 'SAP Order No.', 'Article description'] #exxon
field_lst7 = ['Transport No.', 'SAP Order No.', 'Origin']
field_lst_msd = ['Transport No.', 'SAP Order No.', 'Transport comment:']
field_lst_mitsui = ['Transport No.', 'Transport comment:', 'SAP Order No.', 'Shipping Number (BOL)', 'Article description']
field_lst8 = ['Shipping Number (BOL)', 'Transport No.', 'SAP Order No.']

cust_lst = ['CHAMPIONX SG SERVICE PTE LTD', 'MITSUI ELASTOMERS SINGAPORE PTE LTD', 'MEGACHEM MANUFACTURING PTE LTD', 'CLP INTERNATIONAL PTE LTD', 
            'EXXONMOBIL CHEMICAL COMPANY', 'A.I.M. CHEMICAL INDUSTRIES PTE LTD', 'SUMITOMO CHEMICAL ASIA PTE LTD', 
            'SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG', 'BASF SOUTH EAST ASIA PTE LTD', 'THE POLYOLEFIN - PULAU AYER MERBAU', 
            'MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', 'SINGAPORE ADHESION PLANT', 'ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            'BERTSCHI SINGAPORE PTE. LTD.', 'TOTALENERGIES MARKETING ASIAPACIFIC', 'TOTALENERGIES MARKETING ASIA- PACIFIC MIDDLE EAST PTE LTD (LOBP)',
            'THE POLYOLEFIN - PULAU AYER MERBAU G5', 'THE POLYOLEFIN - PULAU AYER MERBAU G23',
            'SINGAPORE BUTYL PLANT', 'MITSUI ELASTOMERS SINGAPORE', 'CLP INTERNATIONAL']

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)
list_of_pdf = glob.glob(path_pdf +'*.pdf')
for file_pdf in list_of_pdf:
    file = str(file_pdf)
    path_to_sample_documents = file
    with open(path_to_sample_documents, "rb") as f:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-document", document=f, locale="en-US"
        )
        result = poller.result()
        
        lst_trans = []
        for kv_pair in result.key_value_pairs:
            if kv_pair.key and kv_pair.value:
                lst_trans.extend([[kv_pair.key.content, kv_pair.value.content]]) #result.key_value_pairs is list
                #print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
            else:
                #print("Key '{}': Value:".format(kv_pair.key.content))
                print('--------------------')
        analyze_result_dict = result.to_dict()
        
        qq = pd.DataFrame(lst_trans, columns=['SN', 'Va'])
        qqq = qq.set_index('SN').T.reset_index(drop=True)
        
        with open(path_json + 'data' + "_" +str(i)+ ".json", 'w') as f:
            json.dump(analyze_result_dict, f, cls=AzureJSONEncoder)             
        
        if 'Origin' in qqq.columns:
            try:
                q = qqq[field_lst]
            except:
                pass
            try:
                q = qqq[field_lst2]
            except:
                pass
            try:
                q = qqq[field_lst5]
            except:
                pass
            
            try:
                found = list(map(lambda x: x.startswith('BASF SOUTH EAST ASIA'), qqq['Destination']))
                #print(found[0])
                if found[0] == True:
                    try:
                        q = qqq[field_lst4]                     
                    except:
                        q = qqq[field_lst7]
                        q['Cust Ref'] = '0'
            except:
                pass
            try:
                found = list(map(lambda x: x.startswith('TOTALENERGIES'), qqq['Destination']))
                #print(found[0])
                if found[0] == True:
                    try:
                        q = qqq[field_lst1]                     
                    except:
                        pass
            except:
                pass
              
        if 'Origin' not in qqq.columns:
            try:
                q = qqq[field_lst3]
            except:
                pass
            try:
                q = qqq[field_lst4]
            except:
                pass
            try:
                q = qqq[field_lst6]
            except:
                pass
            try:
                q = qqq[field_lst8]
            except:
                pass
            try:
                q = qqq[field_lst_msd] #msd
            except:
                pass
            try:
                found_msd = list(map(lambda x: x.startswith('MEGACHEM'), qqq['Destination']))
                if found_msd[0] == True:
                    try:
                        q = qqq[field_lst6]
                    except:
                        pass
            except:
                pass
            try:
                q = qqq[field_lst_mitsui]
            except:
                pass
        q = q.loc[:, ~q.columns.duplicated()].copy() #remove dup cols
        #q['MV2'] = q['Origin'].map(chawanVopak_dic)
        #q['MV2'] = q['DEPOT_MV2'].apply(lambda x:chawanVopak_dic[x]) #map to new col with dict
        
        try:
            q.rename(columns={'Comment': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True)
        except:
            q.rename(columns={'comment:': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True)
        try:
            q.rename(columns={'Transport comment:': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True)
        except:
            pass
        try:
            q.rename(columns={'Transport comment:': 'Order Remarks', 'Transport No.': 'Transport No', 'Origin': 'DEPOT_MV2', 
                              'SAP Order No.': 'SAP Order No', 'Product Information': 'PRODUCT', 'Shipping Number (BOL)': 'Cust Ref'}, 
                     inplace=True) #basf
        except:
            pass
        
        col_reord = ['Order Remarks', 'Cust Ref', 'Transport No', 'DEPOT_MV2', 'SAP Order No', 'PRODUCT']
        col_reord2 = ['Order Remarks', 'Cust Ref', 'Transport No', 'SAP Order No', 'PRODUCT']
        col_reord3 = ['Cust Ref', 'Transport No', 'SAP Order No', 'PRODUCT'] #exxon
        col_reord4 = ['Transport No', 'DEPOT_MV2', 'SAP Order No'] #basf
        col_reord5 = ['Transport No', 'SAP Order No'] #msd
        col_reord6 = []
        
        if 'Order Remarks' not in q.columns:
            try:
                      new_cols = col_reord3 + (q.columns.drop(col_reord3).tolist())
            except:
                      pass
            try:
                    new_cols = col_reord5 + (q.columns.drop(col_reord5).tolist())
            except:
                    pass
        if 'Order Remarks' in q.columns:
            try:
                new_cols = col_reord + (q.columns.drop(col_reord).tolist())                
            except:
                pass
            try:
                new_cols = col_reord2 + (q.columns.drop(col_reord2).tolist())                
            except:
                    pass
            try:
                new_cols = col_reord5 + (q.columns.drop(col_reord5).tolist())
            except:
                pass
        if ('Order Remarks' not in q.columns) and ('Destination' in qqq.columns):
            try:
                new_cols = col_reord4 + (q.columns.drop(col_reord4).tolist())
            except:
                #new_cols = col_reord3 + (q.columns.drop(col_reord3).tolist())
                new_cols = col_reord5 + (q.columns.drop(col_reord5).tolist())
          
        q = q[new_cols]        
    lst_dfq.append(q)
    i += 1
q_trans = pd.concat(lst_dfq)
#q_trans = q_trans.dropna(subset=['DEPOT_MV2'])
q_trans.drop_duplicates(keep='last', inplace=True)

--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
--------------------
-------------

In [5]:
import numpy as np #Diane
q_trans_2 = q_trans.copy()
#try:
    #q_trans_2['DEPOT_MV2'] = q_trans_2['DEPOT_MV2'].replace(np.nan, 'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal')
#except:
    #pass
col_re = ['Order Remarks', 'Cust Ref', 'Transport No', 'DEPOT_MV2', 'SAP Order No', 'PRODUCT']
col_re_1 = ['Order Remarks', 'Cust Ref', 'Transport No', 'DEPOT_MV2', 'SAP Order No']
col_re_2 = ['Cust Ref', 'Transport No', 'DEPOT_MV2', 'SAP Order No', 'PRODUCT']

try:
    new_col = col_re + (q_trans_2.columns.drop(col_re).tolist())
    q_trans_2 = q_trans_2[new_col]
except:
    pass
try:
    #q_trans_2['Cust Ref'] = ''
    #q_trans_2['PRODUCT'] = ''
    #q_trans_2['Order Remarks'] = ''
    q_trans_2['DEPOT_MV2'] = ''
    new_col_2 = col_re_2 + (q_trans_2.columns.drop(col_re_2).tolist())
    q_trans_2 = q_trans_2[new_col_2]
    #q_trans_2['DEPOT_MV2'] = q_trans_2['DEPOT_MV2'].replace(np.nan, 'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal')
    #q_trans_2['DEPOT_MV2'] = q_trans_2['DEPOT_MV2'].replace('', 'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal')
    new_col = col_re + (q_trans_2.columns.drop(col_re).tolist())
    q_trans_2 = q_trans_2[new_col]
except:
    pass    
#q_trans_2

In [ ]:
[qqq.columns.values] #optional

In [6]:
chawanVopak_dic = {'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant 1 Ayer Chawan Place SG SG-627871 SINGAPORE': 'EXX004',
                   'M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plany 1 Ayer Chawan Place SG': 'EXX004',
                            'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal PENJURU ROAD ': 'EXX003',
                  'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal PENJURU ROAD': 'EXX003',
                  'T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal PENJURU ROAD': 'EXX003',
                  'T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal\nPENJURU ROAD 59\nSG\nSG-609142 SINGAPORE': 'EXX003',
                  'VOPAK': 'EXX003', 'CHAWAN': 'EXX004'}
for i in range(len(q_trans_2)):
    depot_mv2 = q_trans_2.iat[i, 3]
    try:
        depot_mv22 = depot_mv2.split(' ')[1]
        q_trans_2.iat[i, 3] = depot_mv22
    except:
        pass
    #first, last = str(depot_mv2).strip().splitlines()
    #depot_mv22 = depot_mv2.replace(last, '').replace('\n', ' ') 
q_trans_2['MV2'] = q_trans_2['DEPOT_MV2'].map(chawanVopak_dic)

for i in range (len(q_trans_2)):
    
        try:
            q_trans_2.iat[i, 0] = q_trans_2.iat[i, 0].str.split('Vessel</b><br>').str[1]
        except:
            pass
        try:
            q_trans_2.iat[i, 0] = q_trans_2.iat[i, 0].split(' ')[-2:]
        except:
            pass
#q_trans_2

In [7]:
def find_between(s, first, last): #28032023 Diane
    try:
        start = s.index(first) + len(first)
        end = s.index(last, start)
        return s[start:end]
    except ValueError:
        return ""

def find_between_r(s, first, last):
    try:
        start = s.rindex(first) + len(first)
        end = s.rindex(last, start)
        return s[start:end]
    except ValueError:
        return ""
#https://stackoverflow.com/questions/3368969/find-string-between-two-substrings

In [8]:
#parse json, merge df rocks 18/2/2023! Diane
#import json, glob, re, os, shutil
i=0
lst_pp = []
lst_dpp = []
path_json = "C:/Users/john.tan/Documents/Transporeon/Json/"
#path_json = "C:/Users/john.tan/Documents/WMS/Transporeon_split/Json/"
pp = pd.DataFrame(columns=['LOADING_PLANT_MV3', 'Coll Date'])
p_trans = pd.DataFrame(columns=['LOADING_PLANT_MV3', 'Coll Date'])

list_of_json = glob.glob(path_json + '*.json') 
for file_json in list_of_json:
    path_to_json = file_json
    with open(path_to_json) as file_object:
        data = json.load(file_object) #parse json file to Python obj

        z = data['content']
                
        cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
        #print(cust[0])    
        if cust[0] in cust_lst:
            try:
                dest_cx = cust[0]
                print(dest_cx)
                deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #
                deli_date = deli_date[0].split(' ', 1)[0]
                #print(deli_date)
            except:
                pass 
            if len(cust) == 0:
                    z = z.replace('\n', ' ')
                    cust = re.findall(r'(?<=Destination )[^.]*', find_between(z, "SINGAPORE", "SG-"))[0].strip()
                    if cust in cust_lst:
                            dest_cx = cust[0]
                            print(dest_cx)
                            deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #
                            deli_date = deli_date[0].split(' ', 1)[0]
                            #print(deli_date)
           
        if re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
            try:
                first, *others, last = cust[0].strip().splitlines()
                dest_cx = re.search(r"^[^\d]*", last, re.IGNORECASE).group() #get string before digits
                print(dest_cx)
            except:
                pass
        if cust[0] == 'EXXONMOBIL CHEMICAL COMPANY':
            deli_date = deli_date.split('\n')[0]
            #print(deli_date)
        if re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
            try:
                #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
                deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
                deli_datex = deli_date[0].split(' ')[3]
                print(deli_datex)                
            except:
                deli_datex = re.findall(r'(?s)MY-(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
                             
        try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)          
        except:
            pass
        if cust[0] == "CHAMPIONX SG SERVICE PTE LTD":
            try:
                sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
                #sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
                sap_num = re.findall("\d+", sap_num)[0] #get only digits
                print(sap_num)
            except:
                sap_num = re.findall(r'(?s)KGM\nSAP Order No.\n(.*?)\n[^\S\r\n]*Items', z)[0]
                print(sap_num)
        if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
            try:
                sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
                sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
                print(sap_num)                
            except:
                sap_num = re.findall(r'(?s)SAP Order No.(.*?)\n[^\S\r\n]*Items', z)
                sap_num = re.findall(r'(?s)SG(.*?)\n[^\S\r\n]*Delivery No.', sap_num[0], re.MULTILINE)
                #print(sap_num[0].strip())
                first, second, *others = sap_num[0].strip().splitlines()
                sap_num = first
                print(sap_num)
                
        try:
            shipping_num = re.findall(r'(?s)Order type\n(.*?)\n[^\S\r\n]*Shipping ', z)[0] #
            shipping_num = ".".join(re.findall(r'\d+', shipping_num))
        except:
            pass
        try:
            shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1)
        except:
            pass
        if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
            try:
                regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
                shipping_num = regexp.search(z).group(1) #
                shipping_numseika = ".".join(re.findall(r'\d+', shipping_num))
            except: 
                pass
        try:
            basf_prod = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z)[0]
            if (cust[0] == "BASF SOUTH EAST ASIA PTE LTD" and basf_prod !=''):
                shipping_num = ""
        except:
            pass
        if re.search("SINGAPORE ADHESION PLANT", cust[0]):
            try:
                sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
                sap_num = re.findall("\d+", sap_num)[0]
                print(sap_num)              
            except:
                pass
        if re.search("SINGAPORE ADHESION PLANT", cust[0]) and not sap_num[0].isdigit():
            try:
                    z = z.replace('\n', '')
                    patt = r'(?<=\SAP Order No.).+?(?=\Delivery)' #get str between 2 str
                    sap_num_sbp = re.findall(patt, z)
                    sap_num= sap_num_sbp[1]
                    sap_num = sap_num.split('1796ExxonMobil')[0]
                    print(sap_num)
            except:
                    pass
        if re.search("SINGAPORE BUTYL PLANT", cust[0]):
            try:
                sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
                sap_num = re.findall("\d+", sap_num)[0]
            except:
                pass
            try:
                sap_num = re.findall(r'(?s)KGM\nSAP Order No.\n(.*?)\n[^\S\r\n]*Items', z)[0]
                #print(sap_num)
            except:
                pass
                             
            try:
                    z = z.replace('\n', '')
                    patt = r'(?<=\SAP Order No.).+?(?=\Delivery)' #get str between 2 str
                    sap_num_sbp = re.findall(patt, z)
                    sap_num= sap_num_sbp[1]
                    sap_num = sap_num.split('1796ExxonMobil')[0]
                    #print(sap_num)
            except:
                    pass
        if re.search("MITSUI ELASTOMERS SINGAPORE PTE LTD", cust[0]):
            try:
                mit_depot = re.sub('\n', ' ', z)
                depot_mv2 = find_between(mit_depot, "Origin", "SINGAPORE")
            except:
                pass
        try:
            depot_mv2 = re.sub('\n', ' ', z)
            depot_mv2 = find_between(depot_mv2, "Origin", "SINGAPORE")
        except:
            pass
    if (re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]) and deli_date[0].isnumeric() == False):
        try:
            lst_pp.extend([[sap_num, dest_cx, deli_date, shipping_num, depot_mv2]])
        except:
            lst_pp.extend([[sap_num, dest_cx, deli_datex, shipping_num, depot_mv2]])
    if (re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]) and deli_date ==''):
        lst_pp.extend([[sap_num, dest_cx, deli_datex, shipping_num, depot_mv2]])
    elif re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        try:
            deli_date = deli_date[0].split(' ')[0]
            lst_pp.extend([[sap_num, dest_cx, deli_date, shipping_num, depot_mv2]])
        except:
            pass
    elif (re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]) and deli_date.isnumeric() == False):
        try:
            lst_pp.extend([[sap_num, dest_cx, deli_datex, shipping_num, depot_mv2]])
        except:
            pass
    if (re.search("CHAMPIONX SG SERVICE PTE LTD", cust[0]) and deli_date.startswith('SG-')):
        try:
            lst_pp.extend([[sap_num, dest_cx, deli_datex, shipping_num, depot_mv2]])
        except:
            pass

    elif re.search("MITSUI ELASTOMERS SINGAPORE PTE LTD", cust[0]):
        lst_pp.extend([[sap_num, dest_cx, deli_date, shipping_num, depot_mv2]])
    else:
        lst_pp.extend([[sap_num, dest_cx, deli_date, shipping_num, depot_mv2]])   
    
    pp = pd.DataFrame(lst_pp)
    lst_dpp.append(pp)
    #shutil.move(file_json, path_json_1)
p_trans = pd.concat(lst_dpp)

# Rename multiple cols by Index
p_trans.rename(columns={p_trans.columns[0]: 'SAP Order No', p_trans.columns[1]: 'LOADING_PLANT_MV3', p_trans.columns[2]: 'Coll Date', p_trans.columns[3]: 'Cust_Ref', p_trans.columns[4]: 'DEPOT_MV2'}, inplace=True)
#p_trans['Coll Date'] = p_trans['Coll Date'].str.split('\n').str[0]
p_trans['DELIVERY'] = p_trans['Coll Date']  

p_trans[~p_trans.index.duplicated()] #drop dup rows by index
p_trans_repeat = p_trans.copy() #for SAP SBP repeat orders
p_trans.drop_duplicates(subset='SAP Order No', keep='last') #drop dup rows by col#first

ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD
13060370
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000.000
6316564
6316564
13064192
13065022
13065026
4912042569
4912042566
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000.000
6316639
6316639
4912042559
4912042567
4912042554
4912042563
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000.000
6316565
6316565
4912042550
4912042560
4912042561
4912042558
4912042556
4912042552
ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD
13060371
4912042557
4912042545
4912042562
13065027
4912042555
4912042564
4912042553
SINGAPORE ADHESION PLANT
4452584440
4452584440
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000.000
6316563
6316563
CLP INTERNATIONAL PTE LTD
13049549
13064191
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000.000
6316640
6316640
BASF SOUTH EAST ASIA PTE LTD
13056952
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000.000
6316571
6316571
MITSUI ELASTOMERS SINGAPORE PTE LTD
13056702
CHAMPIONX SG SERVICE PTE LTD
10:30
20,000.000
6316567
6316567
MITSUI ELASTOMERS SINGAPORE PTE LTD
13056699
M

,SAP Order No,LOADING_PLANT_MV3,Coll Date,Cust_Ref,DEPOT_MV2,DELIVERY
0,13060370,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,2024-07-19,1004219950,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19
3,6316564,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004219952,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19
4,13064192,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004219206,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19
5,13065022,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004218965,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19
6,13065026,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004218604,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19
7,4912042569,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004218337,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pe...,2024-07-19
8,4912042566,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004218301,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pe...,2024-07-19
11,6316639,CHAMPIONX SG SERVICE PTE LTD,2024-07-23,1004221280,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-23
12,4912042559,CHAMPIONX SG SERVICE PTE LTD,2024-07-23,1004218195,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pe...,2024-07-23
13,4912042567,CHAMPIONX SG SERVICE PTE LTD,2024-07-23,1004218329,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pe...,2024-07-23


In [ ]:
#p_trans.at[33, 'SAP Order No'] = '4452452173' ##
#p_trans.at[1, 'Coll Date'] = '2024-04-06'
#p_trans.at[35, 'Coll Date'] = '2023-10-31'
#p_trans.at[24, 'Coll Date'] = '2023-09-23'
#p_trans.at[51, 'Coll Date'] = '2023-10-03'
#p_trans.at[56, 'Coll Date'] = '2023-10-03'
#p_trans.at[1, 'DELIVERY'] = '2024-04-06'
#p_trans.at[20, 'DELIVERY'] = '2023-11-14'
#p_trans.at[35, 'DELIVERY'] = '2023-10-31'
#p_trans.at[51, 'DELIVERY'] = '2023-10-03'
#p_trans.at[56, 'DELIVERY'] = '2023-10-03'

#p_trans.at[33, 'SAP Order No'] = '0'

#p_trans.drop_duplicates(subset='SAP Order No', keep='last')

In [9]:
df_trans = pd.merge(p_trans, q_trans_2, left_on='SAP Order No', right_on='SAP Order No') #merge on common cols Diane
df_trans.drop_duplicates(subset='SAP Order No', keep='last', inplace=True)
#df_trans[~df_trans.index.duplicated()] #drop dup rows
#print(df_trans)
cols_reo = ['Cust_Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 
            'SAP Order No', 'PRODUCT'] #MV2
cols_reo_1 = ['Cust_Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 
            'SAP Order No', 'PRODUCT']
try:
    df_trans = df_trans[cols_reo]
except:
    pass
try:
    #df_trans['Cust Ref'] = ''
    #df_trans['PRODUCT'] = ''
    df_trans = df_trans[cols_reo_1]
except:
    pass
basf_dict = {'BASF SOUTH EAST ASIA PTE LTD': 'EXXAL 8 ROP VL'} #basf product
basf_dict_lst= list(basf_dict.keys())
df_trans['PRODUCT'] = np.where(df_trans.LOADING_PLANT_MV3==basf_dict_lst[0], 'EXXAL 8 ROP VL', df_trans['PRODUCT'])

df_trans['Cust_Ref'].fillna(0, inplace=True)
df_trans.index = np.arange(1, len(df_trans)+1)

df_trans = df_trans.rename(columns={'DEPOT_MV2_x': 'DEPOT_MV2'})
#df_trans = df_trans.drop(['DEPOT_MV2_y'], axis=1)
#df_trans

In [10]:
from datetime import datetime #1/3/2023! Diane
from datetime import timedelta
df_trans['DELIVERY'] = pd.to_datetime(df_trans['DELIVERY'], format="%Y-%m-%d")
curr_filter = datetime.now() - timedelta(days=1)
df_trans = df_trans.loc[df_trans['DELIVERY'] > curr_filter]
df_trans['DELIVERY'] = df_trans['DELIVERY'].apply(str)

try:
    df_trans.drop(columns=['DEPOT_MV2_y'], axis=1, inplace=True)
except:
    pass

for i in range (len(df_trans)):
    if df_trans.apply(lambda x: (x['Order Remarks'] in ['nan']), axis=1).any() == True:
        df_trans.at[i, 'Order Remarks'] = 'No transports comments'
#df_trans = df_trans.dropna(subset=['Cust Ref'])
#df_trans['DELIVERY']=df_trans['DELIVERY'].str.rsplit(' ', 1, expand=True)[0]
#https://stackoverflow.com/questions/71377372/how-to-extract-text-before-the-last-space-in-a-dataframe-column
#for index, row in df_trans.iterrow():
    #try:
            #df_trans.at[index, 'Order Remarks'] = row['Order Remarks'][1]
    #except:
            #pass
try:
    df_trans['Order Remarks'] = df_trans['Order Remarks'].fillna(df_trans['comment:'])
except:
    pass
df_trans

,SAP Order No,LOADING_PLANT_MV3,Coll Date,Cust_Ref,DEPOT_MV2,DELIVERY,Order Remarks,Cust Ref,Transport No,PRODUCT,comment:,MV2
1,13060370,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,2024-07-19,1004219950,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19 00:00:00,"[TIFU, 1526400]",1004219950,6109997850,EXXSOL HEXANE AP VL,NaN,NaN
2,13064192,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004219206,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19 00:00:00,"[7789390\nPCVU, 2630847]",1004219206,6109992740,SOLVESSO 150 AP VL,NaN,NaN
3,13065022,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004218965,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19 00:00:00,"[General, Text]",1004218965,6109992861,SOLVESSO 150 AP VL,NaN,NaN
4,13065026,CHAMPIONX SG SERVICE PTE LTD,2024-07-19,1004218604,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-19 00:00:00,"[General, Text]",1004218604,6109990603,SOLVESSO 100 AP VL,NaN,NaN
5,6316639,CHAMPIONX SG SERVICE PTE LTD,2024-07-23,1004221280,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-23 00:00:00,"[TIFU7789390,, PCVU2630847]",1004221280,6110006385,SOLVESSO 150 AP VL,NaN,NaN
6,13060371,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,2024-07-23,1004221251,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-23 00:00:00,"[TIFU, 1526400]",1004221251,6110006381,EXXSOL HEXANE AP VL,NaN,NaN
7,13065027,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,2024-07-23,1004219883,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-23 00:00:00,"[TIFU, 7783180\n1525105]",1004219883,6109997854,SOLVESSO 100 AP VL,NaN,NaN
8,4452584440,SINGAPORE ADHESION PLANT,2024-07-17,1004218252,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-17 00:00:00,NaN,1004218252,6109988719,NaN,NaN,NaN
9,13049549,CLP INTERNATIONAL PTE LTD,2024-07-08,1004217866,T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Pe...,2024-07-08 00:00:00,"[PCVU, 2587627]",1004217866,6109987164,ACTREL 3356L AP VL,NaN,NaN
10,13064191,CLP INTERNATIONAL PTE LTD,2024-07-08,1004217794,T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Te...,2024-07-08 00:00:00,"[PCVU, 2630847]",1004217794,6109985846,SOLVESSO 150 AP VL,NaN,NaN


In [11]:
import openpyxl #Diane
import datetime
df_trans['SAP Order No'] = pd.to_numeric(df_trans['SAP Order No'])
df_trans.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
df_trans.index = np.arange(1, len(df_trans)+1)

with pd.ExcelWriter(path_wms + r"For_Diane.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer8:
    sheetName = 'NewTransports_' + datetime.datetime.now().strftime("%Y%m%d %H%M")
    df_trans.to_excel(writer8, sheet_name=sheetName, engine='openpyxl')
    worksheet8 = writer8.sheets[sheetName]
    writer8.save()

C:\Users\john.tan\AppData\Local\Temp\ipykernel_736\3634920575.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer8.save()


In [ ]:
import re #parse json files 23/1/2023! #optional
from re import search

path_dir = path_transporeon + 'Json/'
cols = ['Cust Ref', 'Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'PRODUCT', 'LOADING_PLANT_MV3', 'DELIVERY', 'SAP Order No']
df_trans = pd.DataFrame(columns=cols)
#df_trans.columns = cols
lst_trans = []
i = 0

os.chdir(path_dir)
for file in glob.glob("*.json"):
    file = str(file)
    with open(path_dir+file) as file_object:
        data = json.load(file_object) #parse json file to Python obj

    z = data['content']

    trans_num = re.search(r'Transport No. (.*?) Delivery', z).group(1) #extract str between two strings
    #trans_ID = re.search(r'ID (.*?) Weight', z, flags=re.MULTILINE).group(1)
    #weight = re.search(r'Weight (.*?)kg', z).group(1) #
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    print(cust[0])
    try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)
    except:
            pass
    
    if (cust[0] == 'SINGAPORE ADHESION PLANT' and (isinstance(sap_num, list) == True)):
        try:
                sap_adh = re.findall(r"(\d+)\nOrder type", z)[0]
                print('SAP1')
                print(sap_adh)
        except:
                sap_numbr = re.search('SAP Order No.\n(\d+)', z)
                if sap_numbr:
                    print('SAP2')
                    sap_adh = sap_numbr.group(1)
                    print(sap_adh)           
                
    if cust[0] == "CHAMPIONX SG SERVICE PTE LTD":
        sap_num = re.findall(r'(?s)SG\n(.*?)\n[^\S\r\n]*normalorder', z)[0]
        #sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
        print(sap_num)
    try:
        shipper = re.search(r'(.*?) Scheduling', z).group(1) #
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1) ## get numbers after str

    except:
        shipper = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG', z)[0]       
    try:
        prod_info = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*UTC', z) #
        first, *others, last = prod_info[0].strip().splitlines()
        prod_info = last
        #print(last)
    except:
        prod_info = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z) #
        
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        try:
            #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
            deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
            deli_datex = deli_date[0].split(' ')[3]
            #print(deli_datex)
        except:
            deli_datex = re.findall(r'(?s)MY-(.*?)\n[^\S\r\n]*Measurements', z, re.MULTILINE) #
    
    if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
        try:
            sap_num = re.findall(r'(?s)SAP Order No.\n(.*?)\n[^\S\r\n]*Order type', z)
            sap_num = re.findall("\d+", sap_num[0], re.MULTILINE)[0]
            print(sap_num)
        except:
            sap_num = re.findall(r'(?s)SAP Order No.(.*?)\n[^\S\r\n]*Items', z)
            sap_num = re.findall(r'(?s)SG(.*?)\n[^\S\r\n]*Delivery No.', sap_num[0], re.MULTILINE)
            #print(sap_num[0].strip())
            first, second, *others = sap_num[0].strip().splitlines()
            sap_num = first
            print(sap_num)
        z_msd = z.replace('\n', '')    
        try:
            shipping_numMSD = re.search('SingaporeSG(\d+)', z_msd, re.IGNORECASE).group(1) #
        except:
            pass
        try:
            shipping_numMSD = re.search('\nSG\n(\d+)', z_msd, re.IGNORECASE).group(1)
        except:
            pass
        try:
            shipping_numMSD = shipping_numMSD[-15:]
            shipping_numsd = shipping_numMSD[0:10]
            print('z_msd '+shipping_numsd)
        except:
            pass
        
    if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
        try:
            regexp = re.compile(r'(?s)Order type (.*?)\n[^\S\r\n]*Shipping ', re.MULTILINE)
            shipping_num = regexp.search(z).group(1) #
            shipping_numseika = ".".join(re.findall(r'\d+', shipping_num))
        except: 
            pass
    
    po_num = re.findall(r'(?s)Purchase order No.\n(.*?)\n[^\S\r\n]*Gross Weight', z)
    cust = re.findall(r'(?s)Destination\n(.*?)\n[^\S\r\n]*SG-', z) #MV3
    origin = re.findall(r'(?s)Origin\n(.*?)\n[^\S\r\n]*SG-', z) #MV2
    plant = re.findall(r'(?s)Division\n(.*?)\n[^\S\r\n]*Product', z)
    #vehicle = re.search(r'AP - (.*?) T', z).group(1)
    #vehicle = str(vehicle) + 'Truck'
    
    #deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]* ', z)
    deli_date = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #
    deli_date = deli_date[0].split(' ', 1)[0]
    if cust[0] == 'EXXONMOBIL CHEMICAL COMPANY':
        deli_date = deli_date.split('\n')[0]
        
    if cust[0] == 'RESPACK MANUFACTURING SDN BHD':
        p = re.compile('MY-([\S\s]*)\n', re.MULTILINE) #Respack
        deli_date = re.findall(p, respack)[0].split('\n')[1].split(' ')[0]
    try:
        shipping_num = re.findall(r'(?s)Order type\n(.*?)\n[^\S\r\n]*Shipping ', z)[0] #
        shipping_num = ".".join(re.findall(r'\d+', shipping_num))
    except:
        pass
    try:
        shipping_num = re.search('\nnormalorder\n(\d+)', z, re.IGNORECASE).group(1)
    except:
        pass
    prod_info = re.findall(r'(?s)Product information\n(.*?)\n[^\S\r\n]*Loading', z) #
    
    try:
        deadline = re.search(r'Deadline (.*?) ', z).group(1)
        tifu_patt = r'(?<=General Text)(\s.+)?\nAdditional'
        tifu = re.findall(tifu_patt, z, re.DOTALL)[0]
        tifu = os.linesep.join([s for s in tifu.splitlines() if s])
        first, *others = tifu.splitlines()
    except:
        pass  
    try:
        ord_rem = cust[0] + prod_info[0]
    except:
        pass
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        z_cx = z.replace('\n', '')
        try:
            prod_info =  re.search(r'Article description(.*?)Comment', z_cx).group(1) ##regex between 2 strings
            ord_rem = cust[0]+prod_info
        except:
            prod_info = re.search(r'Article short description(.*?)Article description', z_cx) #
    if search('BASF SOUTH EAST ASIA PTE LTD', cust[0]):
        z_basf = z.replace('\n', '')
        try:
            prod_info =  re.search(r'Article description(.*?)Comment', z_basf).group(1) ##regex between 2 strings
            print('basf prod')
            print(prod_info)
            ord_rem = cust[0] + prod_info
        except:
            prod_info = re.search(r'Product information(.*?)Loading time', z_basf) # 
            try:
                ord_rem = cust[0] + prod_info
            except:
                pass
    
    if origin[0] =='M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant':
            df_trans.loc[i, 'MV2'] = 'EXX004'
    else:
        df_trans.loc[i, 'MV2'] = 'EXX003'
    if cust[0] == 'THE POLYOLEFIN - PULAU AYER MERBAU':
        try:
            cust_1 = cust[1].split('\n')[-1].split(', SG')[0]
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
        try:
            cust_1 = re.findall(r'(?s)SINGAPORE\n(.*?)\n[^\S\r\n]*Measurements', z) #THE POLYOLEFIN - PULAU AYER MERBAU
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust_1
        except:
            pass
    else: 
        df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    
    try:
        df_trans.loc[i, 'Cust Ref'] = shipping_num
    except:
        pass
    try:
        df_trans.loc[i, 'Cust Ref'] = second
    except:
        pass
    if cust[0] == "MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)":
        try:
            df_trans.loc[i, 'Cust Ref'] = shipping_numsd
        except:
            df_trans.loc[i, 'Cust Ref'] = shipping_num
    
    if cust[0] == "SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG":
        try:
            df_trans.loc[i, 'Cust Ref'] = shipping_numseika
        except:
            df_trans.loc[i, 'Cust Ref'] = shipping_num
    try:
        df_trans.loc[i, 'Order Remarks'] = ord_rem
    except:
        pass
    df_trans.loc[i, 'Transport No'] = trans_num
    df_trans.loc[i, 'Coll Date'] = deli_date
    df_trans.loc[i, 'DEPOT_MV2'] = origin[0]
    try:
        df_trans.loc[i, 'PRODUCT'] = prod_info[0]
    except:
        df_trans.loc[i, 'PRODUCT'] = last
    if search("CHAMPIONX SG SERVICE PTE LTD", cust[0]):
        df_trans.loc[i, 'PRODUCT'] = prod_info
    
    df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[0]
    df_trans.loc[i, 'DELIVERY'] = deli_date
    try:
        df_trans.loc[i, 'SAP Order No'] = sap_num
    except:
        pass
    if (cust[0] == 'SINGAPORE ADHESION PLANT' and (isinstance(sap_num, list) == True)):
        try:
            df_trans.loc[i, 'SAP Order No'] = sap_adh
        except:
            pass
    try:
        if cust[1] =="CHAMPIONX SG\nSERVICE PTE LTD":
            df_trans.loc[i, 'DELIVERY'] = deli_datex
            df_trans.loc[i, 'Coll Date'] = deli_datex
            df_trans.loc[i, 'LOADING_PLANT_MV3'] = cust[1].replace('\n', ' ')
            df_trans.loc[i, 'DEPOT_MV2'] = origin[0].replace('Destination\n', '')
    except: pass
    i += 1

df_trans

In [ ]:
df_trans_trans = df_trans.copy() #optional

df_trans_trans['Coll Date'] = [x.split('\n')[0].replace(' ', '') for x in df_trans_trans['Coll Date']]
#df_trans_trans['DELIVERY'] = [x.split('\n')[0].replace(' ', '') for x in df_trans_trans['DELIVERY']]
df_trans_trans['PRODUCT'] = [x.split('\n')[0].replace(' ', '') for x in df_trans_trans['PRODUCT']]
#s = '(2023-02-03\n18,000)'
#m = re.match('\(([\w\s]+)\n([\w\s]+)\)', s)
#coll_date = m.group(1)
#tim = m.group(2)
df_trans_trans['DELIVERY'] = df_trans_trans['DELIVERY'].str[0: -8]
df_trans = df_trans_trans.copy()
df_trans
#https://stackoverflow.com/questions/44922259/python-using-regex-to-extract-parts-of-a-string-in-pandas-column

In [ ]:
basf_dict = {'BASF SOUTH EAST ASIA PTE LTD': 'EXXAL 8 ROP VL'} #Optional
basf_dict_lst= list(basf_dict.keys())
df_trans['PRODUCT'] = np.where(df_trans.LOADING_PLANT_MV3==basf_dict_lst[0], 'EXXAL 8 ROP VL', df_trans.PRODUCT)

df_trans['Cust Ref'].fillna(0, inplace=True)
#df_trans['LOADING_PLANT_MV3'].isin(basf_dict_lst)
df_trans.head()

In [12]:
temp = pd.read_csv(path_transporeon + r"Template_Local.csv") #template for upload Diane
tables = pd.read_csv(path_transporeon + r"Tables.csv") #tables for mapping

In [13]:
temp_upload = pd.concat([temp, df_trans], ignore_index=True) #Diane
#print(temp_upload)
#temp_upload = pd.merge(left=temp_upload, right=tables, how='left') #merge df 
temp_upload = temp_upload.fillna(method='ffill')
#temp_upload = temp_upload.drop(['MV3'], axis=1)
temp_upload = temp_upload.rename(columns={'Code': 'MV3'}) #for upload to TMS

In [14]:
#Diane
colss = ['Customer No', 'Order Remarks', 'Delivery Type No', 'ISO abbreviation', 'Cargo Type No', 'MV1', 'MV2', 'MV4', 'MV5', 'MV6',
         'Plant Code', 'Special Handling No', 'Delivery Type No']
temp_upload_1 = temp_upload.drop(columns=colss, axis=1)
temp_upload_1 = temp_upload_1.rename(columns={'LOADING_PLANT_MV3': 'LOADING PLANT', 'DEPOT_MV2': 'DEPOT'})
colss_reord = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'SAP Order No']

temp_upload_1 = temp_upload_1[colss_reord] #to compare with master
temp_upload_1 = temp_upload_1.reset_index().drop(columns='index')
temp_upload_1['SAP Order No'] = temp_upload_1['SAP Order No'].astype(np.int64)
#temp_upload_1

In [15]:
temp_upload_1['Coll Date'] = pd.to_datetime(temp_upload_1['Coll Date']).dt.strftime('%Y-%m-%d') #format datetime Diane
temp_upload_1['DELIVERY'] = pd.to_datetime(temp_upload_1['DELIVERY']).dt.strftime('%Y-%m-%d')

#dict
depot_dic = {' M1796 CHAWAN FLUID PLANT SG10 PAC - Chawan Fluid Plant SG-627871 ': 'Chawan Fluid Plant \n1 Ayer Chawan Place',
             ' T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal SG-609142 ': 'Vopak (Penjuru) Terminal \n PENJURU ROAD 59',
             ' T1796 VOPAK TERMINAL - SINGAPO SG01 Vopak (Penjuru) Terminal SG-609142 ': 'Vopak (Penjuru) Terminal \n PENJURU ROAD 59',
             ' Destination T1796 VOPAK (PENJURU) SG01 Vopak (Penjuru) Terminal PENJURU ROAD 59 SG CHAMPIONX SG SERVICE PTE LTD 11 AYER MERBAU ROAD, JURONG ISLAND SG-627526 ': 'Vopak (Penjuru) Terminal \n PENJURU ROAD 59'
             }
temp_upload_1['DEPOT'] = temp_upload_1['DEPOT'].map(depot_dic)

temp_upload_1['SAP Order No'] = pd.to_numeric(temp_upload_1['SAP Order No'])
#temp_upload_1.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
temp_upload_1

,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1004219251,6109992739,2024-07-18,Vopak (Penjuru) Terminal \n PENJURU ROAD 59,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2024-07-18,6316566
2,1004221280,6110006385,2024-07-23,Vopak (Penjuru) Terminal \n PENJURU ROAD 59,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2024-07-23,6316639
3,1004221270,6110006384,2024-07-23,Vopak (Penjuru) Terminal \n PENJURU ROAD 59,SOLVESSO 150 AP VL,CHAMPIONX SG SERVICE PTE LTD,2024-07-23,6316640
4,1004217866,6109987164,2024-07-08,Vopak (Penjuru) Terminal \n PENJURU ROAD 59,ACTREL 3356L AP VL,CLP INTERNATIONAL PTE LTD,2024-07-08,13049549
5,1004219882,6109997548,2024-07-19,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2024-07-19,13056699
6,1004219957,6109997549,2024-07-19,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2024-07-19,13056700
7,1004221201,6110006378,2024-07-23,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI ELASTOMERS SINGAPORE PTE LTD,2024-07-23,13056702
8,1004219195,6109992735,2024-07-18,Vopak (Penjuru) Terminal \n PENJURU ROAD 59,EXXAL 8 ROP VL,BASF SOUTH EAST ASIA PTE LTD,2024-07-18,13056951
9,1004221139,6110004559,2024-07-22,Vopak (Penjuru) Terminal \n PENJURU ROAD 59,EXXAL 8 ROP VL,BASF SOUTH EAST ASIA PTE LTD,2024-07-22,13056952
10,1004219950,6109997850,2024-07-19,Vopak (Penjuru) Terminal \n PENJURU ROAD 59,EXXSOL HEXANE AP VL,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,2024-07-19,13060370


In [16]:
columns = ['NO', 'TIME SLOT', 'REMARKS'] #Diane SH Reference Shipper Permit ETA Date QTY Cont Size Cont Typen Cont No SEAL
master = pd.read_csv(path_transporeon + r"EM master file.csv")
master = master.drop(columns=columns, axis=1)
master = master.iloc[1:] #remove first row
master = master.reset_index().drop(columns='index')

from datetime import datetime
try:
    master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') #format datetime
    master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
except:
    pass

master.rename(columns={'SAP no': 'EM SO# Ref', 'Cust Ref (SHIPPING NUMBER-BOL)': 'Cust Ref', 'DESTINATION': 'LOADING PLANT'}, 
inplace=True) #13072023

master.drop(index=master.index[0], axis=1, inplace=True) #drop first row
master['EM SO# Ref'] = pd.to_numeric(master['EM SO# Ref'], errors='coerce') #convert non-numeric values to NaN
master.dropna(subset=['EM SO# Ref'], inplace=True) #drop rows with NaN in col
master['EM SO# Ref'] = master['EM SO# Ref'].astype(np.int64)
#master['EM SO# Ref'] = master['EM SO# Ref'].apply(pd.to_numeric)

C:\Users\john.tan\AppData\Local\Temp\ipykernel_736\3957656296.py:2: DtypeWarning: Columns (0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  master = pd.read_csv(path_transporeon + r"EM master file.csv")


In [17]:
temp_upload_cust = temp_upload_1['SAP Order No'].tolist() #Diane
temp_upload_cust = list(map(int, temp_upload_cust)) #convert list items to int

master = master[master['EM SO# Ref'].isin(temp_upload_cust)] #select rows based on list
#master = master[master['SAP Order No'].isin(temp_upload_cust)]
col_mas = ['Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT', 'LOADING PLANT', 'DELIVERY', 'EM SO# Ref'] #'EM SO# Ref'
master = master[col_mas]
master.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)

master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master.index = np.arange(1, len(master)+1)

#master.loc[~master['Cust Ref'].str.contains('[a-zA-Z]')]
master = master[~master['Cust Ref'].isin(['TBA', 'tba'])]
master = master[~master['Coll Date'].isin(['HOLD'])] #remove rows

master['Cust Ref'].fillna(value=0, inplace=True)
master['Transport No'].fillna(value=0, inplace=True)
try:
    master['Cust Ref'] = master['Cust Ref'].astype('int64') #
    master['Transport No'] = master['Transport No'].astype('int64')
except Exception as e:
    print(e)

if len(master) == 0:
    print('\033[1m'+ '\033[94m'+ 'All download Transports are new')

master.dropna(subset=['Coll Date', 'DELIVERY'], inplace=True)
master.loc[~master['Coll Date'].str.contains('[a-zA-Z]')] #remove rows with alphabets in col
master.loc[~master['DELIVERY'].str.contains('[a-zA-Z]')]
master
#https://stackoverflow.com/questions/69255495/remove-rows-with-alphabets-in-a-given-column


invalid literal for int() with base 10: '1004212525 (new TD:1004217866)'


,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1004219251,6109992739,18/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",18/7/2024,6316566
2,1004208743,6109919791,26/6/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,26/6/2024,13049549
3,1004212525 (new TD:1004217866),6109927018 (new: 6109987164),4/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,8/7/2024,13049549
4,1004217866,6109987164,8/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,8/7/2024,13049549
5,1004219882,6109997548,19/7/2024,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,19/7/2024,13056699
6,1004219957,6109997549,19/7/2024,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,19/7/2024,13056700
7,1004219195,6109992735,18/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,18/7/2024,13056951
9,1004219950,6109997850,19/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL HEXANE AP VL,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,19/7/2024,13060370
10,1004217794,6109985846,11/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,MEGACHEM MANUFATURING\n11 TUAS LINK 1,11/7/2024,13064191
11,1004219206,6109992740,18/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,MEGACHEM MANUFATURING\n11 TUAS LINK 1,18/7/2024,13064192


In [18]:
master.drop([3])


,Cust Ref,Transport No,Coll Date,DEPOT,PRODUCT,LOADING PLANT,DELIVERY,SAP Order No
1,1004219251,6109992739,18/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",18/7/2024,6316566
2,1004208743,6109919791,26/6/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,26/6/2024,13049549
4,1004217866,6109987164,8/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,8/7/2024,13049549
5,1004219882,6109997548,19/7/2024,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,19/7/2024,13056699
6,1004219957,6109997549,19/7/2024,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,19/7/2024,13056700
7,1004219195,6109992735,18/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXAL 8 ROP VL,BASF 20 BANYAN AVENUE JURONG ISLAND,18/7/2024,13056951
9,1004219950,6109997850,19/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL HEXANE AP VL,ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD,19/7/2024,13060370
10,1004217794,6109985846,11/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,MEGACHEM MANUFATURING\n11 TUAS LINK 1,11/7/2024,13064191
11,1004219206,6109992740,18/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,MEGACHEM MANUFATURING\n11 TUAS LINK 1,18/7/2024,13064192
12,1004218965,6109992861,12/7/2024,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"AFTON CHEMICAL ASIA PTE. LTD\nSERAYA RISE, JUR...",12/7/2024,13065022


In [19]:
master = master[~master['DELIVERY'].isin(['CANCEL', 'ON-HOLD'])] #25052023 Diane

if len(master) != 0:
    try:
        master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d') #format datetime
        master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
    except:
        pass
    try:
        master['Coll Date'] = pd.to_datetime(master['Coll Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
        master['DELIVERY'] = pd.to_datetime(master['DELIVERY'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
    except:
        pass     
master_comp_col = master.copy()

In [20]:
from datetime import timedelta #Diane
columns = ['NO',  'TIME SLOT', 'REMARKS'] #'TD- STATUS' 'SH Reference' 'Shipper', 'Permit', 'ETA Date', 'QTY', 'Cont Size', 'Cont Type', 'Cont No', 'SEAL'
df_sap = pd.read_csv(path_transporeon + r"EM master file.csv")
df_sap = df_sap[~df_sap['Coll Date'].isin(['HOLD'])]
#df_sap.rename(columns={'EM SO# Ref (SAP ORDER NO)': 'EM SO# Ref', 'Cust Ref (SHIPPING NUMBER-BOL)': 'Cust Ref'},
# inplace=True)
#df_sap = master.copy() #transports with changes
df_sap = df_sap.drop(columns=columns, axis=1)
df_sap = df_sap.reset_index().drop(columns='index')

df_sap.rename(columns={'EM SO# Ref': 'SAP_Order_No'}, inplace=True)
sap = df_sap[df_sap.duplicated('SAP_Order_No', keep=False)].groupby('SAP_Order_No')['Coll Date'].apply(list).reset_index()
lst_sap = sap['SAP_Order_No'].tolist()
sap_1 = df_sap[df_sap["SAP_Order_No"].isin(lst_sap)]

curr = datetime.now()
coll_start = curr - timedelta(days=30)
#coll_end = curr + timedelta(days=14)

from simple_colors import *
if len(lst_sap) != 0:
    for i in range(len(sap)):
        coll_date = sap.at[i, 'Coll Date']
        coll_date_lst = [datetime.strptime(coll, "%d/%m/%Y").date() for coll in coll_date]
        coll_latest = max(coll_date_lst)#.strftime("%Y-%m-%d")
        
        if coll_latest > coll_start.date():
            print('\033[1m'+ '\033[34m' + 'List of Coll Date')
            print('SAP Order Number: '+ magenta(str(lst_sap[i]).strip(), ['bold', 'underlined'])+' '+green(str(max(coll_date_lst).strftime("%d/%m/%Y")), ['bold', 'underlined']))
            print(sap.at[i, 'Coll Date'])       
else:
    print('No transports changes')

C:\Users\john.tan\AppData\Local\Temp\ipykernel_736\1636922931.py:3: DtypeWarning: Columns (0,1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sap = pd.read_csv(path_transporeon + r"EM master file.csv")


List of Coll Date
SAP Order Number: 6282056.0 14/06/2024
['15/5/2024', '12/6/2024', '14/6/2024']
List of Coll Date
SAP Order Number: 6282057.0 25/06/2024
['17/5/2024', '13/6/2024', '25/6/2024']
List of Coll Date
SAP Order Number: 6294339.0 01/07/2024
['19/6/2024', '27/6/2024', '1/7/2024']
List of Coll Date
SAP Order Number: 6294340.0 01/07/2024
['19/6/2024', '27/6/2024', '1/7/2024']
List of Coll Date
SAP Order Number: 13029053.0 21/06/2024
['3/6/2024', '21/6/2024']


ValueError: time data '13/06/224' does not match format '%d/%m/%Y'

In [ ]:
from datetime import datetime #Diane
for i in range(len(sap)):
    sap_order = sap.at[i, 'SAP_Order_No']
    coll_date = sap.at[i, 'Coll Date']
    coll_date_lst = [datetime.strptime(coll, "%d/%m/%Y").date() for coll in coll_date] #convert elem in list to date
    print('SAP Order Number: '+ str(sap_order).strip() + ' Latest update coll date: '+ max(coll_date_lst).strftime("%Y-%m-%d"))

In [21]:
df_sap_1 = df_sap.drop(df_sap.iloc[:, 9:15], axis=1) #Diane
df_sap_1.drop(df_sap_1.columns[0], axis=1, inplace=True) #drop first col


#df_sap_1[['SAP_Order_No', 'SAP_Order_No1']] = df_sap_1['SAP_Order_No'].str.split(' / ', 1, expand=True) #split col in 2
#df_sap_1 = df_sap_1.iloc[:, :-1] #drop last col
df_sap_1['SAP_Order_No'] = pd.to_numeric(df_sap_1['SAP_Order_No'], errors='coerce') #convert non-numeric values to NaN
df_sap_1.dropna(subset=['SAP_Order_No'], inplace=True) #drop rows with NaN in col

In [22]:
#new transports works 02/02/2023! Diane
uploaded_trans = df_trans['SAP Order No'].tolist()
master_trans = df_sap_1['SAP_Order_No'].tolist()

uploaded_trans = np.unique(uploaded_trans).astype(float)
master_trans = np.unique(master_trans).astype(float)

#np.setdiff1d(uploaded_trans, master_trans).astype(int)
uploaded_trans = [x for x in uploaded_trans if x not in master_trans]
uploaded_trans = list(map(int, uploaded_trans)) #convert to int
uploaded_trans = [ele for ele in uploaded_trans if ele > 0]

if len(uploaded_trans) !=0:
    print('New transport SAP Order Number ')
    print('\033[1m'+ '\033[32m'+ str(uploaded_trans))
else:
    print('No new SAP Order Number')

new_trans = [] #convert list of int to str
for x in uploaded_trans:
    str_x = "% s" % x
    new_trans.append(str_x)

df_newtrans = df_trans[df_trans['SAP Order No'].isin(new_trans)]

df_newtrans.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
if len(df_newtrans) !=0:
    df_newtrans

New transport SAP Order Number 
[6316639, 6316640, 13056702, 13060371]


C:\Users\john.tan\AppData\Local\Temp\ipykernel_736\554645104.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_newtrans.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')


In [23]:
temp_upload_1['Cust Ref'] = temp_upload_1['Cust Ref'].astype(np.int64) #Diane
temp_upload_1['Transport No'] = temp_upload_1['Transport No'].astype(np.int64)
temp_upload_5 = temp_upload_1.copy()
#temp_upload_5.rename(columns={'Cust Ref': 'Cust Ref'}, inplace=True)

In [ ]:
#comp (iterate loading plant) with color, write to excel works 20/1/2023! optional
#import numpy as np

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

def repeatRows(d, n):
    return pd.concat([d]*n)

temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
            
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #print(df_comp)
        with pd.ExcelWriter(path_transporeon + r"Output/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+ str(i) + datetime.now().strftime("%Y%m%d_%H%M"), engine='openpyxl')
        with pd.ExcelWriter(path_wms + r"comp.xlsx", mode="a", engine="openpyxl") as writer1:
            df_comp.to_excel(writer1, sheet_name='comp_'+ str(i) + datetime.now().strftime("%Y%m%d_%H%M"), engine='openpyxl')

In [ ]:
class MyList(list):
    def __contains__(self, typ):
        return any(isinstance(val, typ) for val in self) #optional

x = MyList(uploaded_trans)
print (int in x) #optional optional

In [ ]:
#optional
def repeatRows(d, n):
    return pd.concat([d]*n)

n = abs(len(master) - len(temp_upload_1)) #ensure both df same len
if n!= 0:
    if len(master) > len(temp_upload_1):
        temp_upload_1 = pd.concat([temp_upload_1, repeatRows(temp_upload_1[-1:], n)])
        temp_upload_1.index = np.arange(1, len(temp_upload_1)+1)
        #print(len(temp_upload_1))
    else:
        master = pd.concat([master, repeatRows(master[-1:], n)])
        master.index = np.arange(1, len(master)+1)
        #print(len(master))
else:
    print('No existing transports to compare')

In [ ]:
df_all = pd.concat([temp_upload_1, master], axis='columns', keys=['First', 'Second']) #optional
df_comp = df_all.swaplevel(axis='columns')[temp_upload_1.columns[1:]]

def highlight_diff(data, color='lightgreen'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

master = master.dropna(subset=['Transport No','Coll Date'], how='all')
temp_upload_1 = temp_upload_1.dropna(subset=['Transport No','Coll Date'], how='all')

df_comp = df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
#df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)
df_comp

In [ ]:
master.compare(temp_upload_1, keep_shape=True, keep_equal=True) #optional

In [24]:
#comp diffs 29/03/2023 Diane
#df_b = pd.DataFrame({
    #'LOADING_PLANT_MV3': ['VOPAK', 'CHAWAN', 'VOPAK', 'VOPAK'],
    #'Cust Ref': ['1003123', '1003325', '1003000', '1003689'], 
    #'SAP Order No': ['596600', '563322', '560011', '126379'],
    #'DELIVERY': ['2023-03-09', '2023-03-12', '2023-03-14', '2023-03-08'],
    #'PRODUCT': ['EXXAL ROP 8', 'SOLVESSO AL VL', 'EXXAL ROP 8', 'SPECTRA AL VL']
#})
def getColChangedName(row):
    colsChanged = []
    for c in master_comp_col.columns.values:
        if row[c+"_diff"] == True:
            colsChanged.append(c)
    return ", ".join(colsChanged)

master_comp_col = master_comp_col.set_index("SAP Order No")
#master = master.set_index("SAP Order No")
temp_upload_5 = temp_upload_5.set_index("SAP Order No")

diffs = master_comp_col.merge(temp_upload_5, left_index=True, right_index=True, suffixes=('_old', '_new'))

diff_cols = [c+"_diff" for c in master_comp_col.columns.values]

for c in master_comp_col.columns.values:
    diffs[c+"_diff"] = diffs[c+"_old"] != diffs[c+"_new"]
    
diffs["Record_Changed"] = diffs[diff_cols].sum(axis=1)/len(diff_cols) > 0

diffs = diffs[diffs["Record_Changed"] == 1]
diffs["Cols_Changed"] = diffs.apply(lambda row: getColChangedName(row), axis=1)
diffs.head()
#https://stackoverflow.com/questions/70550234/compare-2-pandas-dataframe-with-different-lengths-and-get-the-column-name-which

,Cust Ref_old,Transport No_old,Coll Date_old,DEPOT_old,PRODUCT_old,LOADING PLANT_old,DELIVERY_old,Cust Ref_new,Transport No_new,Coll Date_new,...,DELIVERY_new,Cust Ref_diff,Transport No_diff,Coll Date_diff,DEPOT_diff,PRODUCT_diff,LOADING PLANT_diff,DELIVERY_diff,Record_Changed,Cols_Changed
SAP Order No,,,,,,,,,,,,,,,,,,,,,
6316566,1004219251,6109992739,2024-07-18,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"CHAMPION X 11 AYER MERBAU ROAD, JURONG ISLAND",2024-07-18,1004219251,6109992739,2024-07-18,...,2024-07-18,True,True,False,True,True,True,False,True,"Cust Ref, Transport No, DEPOT, PRODUCT, LOADIN..."
13049549,1004208743,6109919791,2024-06-26,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,2024-06-26,1004217866,6109987164,2024-07-08,...,2024-07-08,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
13049549,1004212525 (new TD:1004217866),6109927018 (new: 6109987164),2024-07-04,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,2024-07-08,1004217866,6109987164,2024-07-08,...,2024-07-08,True,True,True,True,True,True,False,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
13049549,1004217866,6109987164,2024-07-08,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,2024-07-08,1004217866,6109987164,2024-07-08,...,2024-07-08,True,True,False,True,True,True,False,True,"Cust Ref, Transport No, DEPOT, PRODUCT, LOADIN..."
13056699,1004219882,6109997548,2024-07-19,Chawan Fluid Plant \n1 Ayer Chawan Place,EXXSOL HEXANE SP AP VL,MITSUI 701 AYER MERBAU ROAD,2024-07-19,1004219882,6109997548,2024-07-19,...,2024-07-19,True,True,False,False,False,True,False,True,"Cust Ref, Transport No, LOADING PLANT"


In [25]:
lst_diff_select = [True] #Diane
diffs_1 = diffs.loc[diffs['DELIVERY_diff'].isin(lst_diff_select)] #select rows where col value is iterable
#diffs_1 = diffs.loc[diffs['DELIVERY_diff'] == True]
diffs_1

,Cust Ref_old,Transport No_old,Coll Date_old,DEPOT_old,PRODUCT_old,LOADING PLANT_old,DELIVERY_old,Cust Ref_new,Transport No_new,Coll Date_new,...,DELIVERY_new,Cust Ref_diff,Transport No_diff,Coll Date_diff,DEPOT_diff,PRODUCT_diff,LOADING PLANT_diff,DELIVERY_diff,Record_Changed,Cols_Changed
SAP Order No,,,,,,,,,,,,,,,,,,,,,
13049549,1004208743,6109919791,2024-06-26,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,ACTREL 3356 L FLUID,CLP INRERNATIONAL\n2 TANJONG PENJURU,2024-06-26,1004217866,6109987164,2024-07-08,...,2024-07-08,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
13064191,1004217794,6109985846,2024-07-11,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,MEGACHEM MANUFATURING\n11 TUAS LINK 1,2024-07-11,1004217794,6109985846,2024-07-08,...,2024-07-08,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
13064192,1004219206,6109992740,2024-07-18,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,MEGACHEM MANUFATURING\n11 TUAS LINK 1,2024-07-18,1004219206,6109992740,2024-07-19,...,2024-07-19,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
13065022,1004218965,6109992861,2024-07-12,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 150 AP VL,"AFTON CHEMICAL ASIA PTE. LTD\nSERAYA RISE, JUR...",2024-07-12,1004218965,6109992861,2024-07-19,...,2024-07-19,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
13065026,1004218604,6109990603,2024-07-15,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 100 AP VL,"AFTON CHEMICAL ASIA PTE. LTD\nSERAYA RISE, JUR...",2024-07-15,1004218604,6109990603,2024-07-19,...,2024-07-19,True,True,True,True,False,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, LOAD..."
13065027,1004219883,6109997854,2024-07-19,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,SOLVESSO 100 AP VL,"AFTON CHEMICAL ASIA PTE. LTD\nSERAYA RISE, JUR...",2024-07-19,1004219883,6109997854,2024-07-23,...,2024-07-23,True,True,True,True,False,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, LOAD..."
4452584440,1004217345,6109982680,2024-07-18,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL D40 AP VL,SINGAPORE ADHESION PLANT 100 JURONG ISLAND HIG...,2024-07-18,1004218252,6109988719,2024-07-17,...,2024-07-17,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
4452584440,1004216270,6109974421,2024-07-10,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL D40 AP VL,SINGAPORE ADHESION PLANT 100 JURONG ISLAND HIG...,2024-07-10,1004218252,6109988719,2024-07-17,...,2024-07-17,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."
4452584440,1004218252,6109988719,2024-07-19,Vopak (Penjuru) Terminal \nPENJURU ROAD 59,EXXSOL D40 AP VL,SINGAPORE ADHESION PLANT 100 JURONG ISLAND HIG...,2024-07-19,1004218252,6109988719,2024-07-17,...,2024-07-17,True,True,True,True,True,True,True,True,"Cust Ref, Transport No, Coll Date, DEPOT, PROD..."


In [ ]:
with pd.ExcelWriter(path_transporeon + "Output/ForDiane_Diffs.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer3:
    sheetName = 'Diffs_' + datetime.now().strftime("%Y%m%d %H%M")
    diffs_1.to_excel(writer3, sheet_name=sheetName, engine='openpyxl')
    worksheet3 = writer3.sheets[sheetName]
    writer3.save()

In [26]:
trans_1 = df_trans[df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)] #Diane
#trans_1 = df_trans[~df_trans['DEPOT_MV2'].str.contains('VOPAK', na=False)]
vopak = trans_1.copy()
#vopak = vopak.iloc[:, 1:] #drop first col
vopak.reset_index(drop=True, inplace=True)

#vopak.insert(loc=0, column='Transporter Name', value='SH COGENT LOGISTICS PTE LTD')
#vopak.insert(loc=0, column='Customer Name', value='ExxonMobil Chemical Asia Pacific')

vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY'])
#vopak = vopak.drop(columns=['Order Remarks', 'Transport No', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'MV2'])
#vopak.rename(columns={'Cust Ref': 'Customer Reference Number', 'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)
vopak.rename(columns={'Coll Date': 'Schedule Date', 'PRODUCT': 'Product name'}, inplace=True)
drop_col = ['Cust_Ref', 'comment:', 'MV2'] #drop cols by list
filter_col = vopak.filter(drop_col)
vopak.drop(filter_col, inplace=True, axis=1)
#df_trans = pd.read_csv(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/temp_upload.csv") #Textract

In [27]:
vopak_SheetNames = pd.ExcelFile(path_transporeon+ r"Output/ForDiane_Vopak.xlsx").sheet_names #Diane
yesterday = max([pd.to_datetime(elem.split('_')[1]) for elem in vopak_SheetNames]).strftime('%Y%m%d') 
yesterday

'20240705'

In [28]:
with pd.ExcelWriter(path_transporeon + "Output/ForDiane_Vopak.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer4:
    sheetName = 'Vopak_' + datetime.now().strftime("%Y%m%d")
    vopak.to_excel(writer4, sheet_name=sheetName, engine='openpyxl')
    worksheet4 = writer4.sheets[sheetName]
    writer4.save()
vopak

C:\Users\john.tan\AppData\Local\Temp\ipykernel_736\1946566366.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer4.save()


,SAP Order No,Schedule Date,Cust Ref,Product name
0,6316566,2024-07-18,1004219251,SOLVESSO 150 AP VL
1,6316639,2024-07-23,1004221280,SOLVESSO 150 AP VL
2,6316640,2024-07-23,1004221270,SOLVESSO 150 AP VL
3,13049549,2024-07-08,1004217866,ACTREL 3356L AP VL
4,13056951,2024-07-18,1004219195,EXXAL 8 ROP VL
5,13056952,2024-07-22,1004221139,EXXAL 8 ROP VL
6,13060370,2024-07-19,1004219950,EXXSOL HEXANE AP VL
7,13060371,2024-07-23,1004221251,EXXSOL HEXANE AP VL
8,13064191,2024-07-08,1004217794,SOLVESSO 150 AP VL
9,13064192,2024-07-19,1004219206,SOLVESSO 150 AP VL


In [29]:
#vopak_SheetNames = pd.ExcelFile(path_transporeon+ r"Output/ForDiane_Vopak.xlsx").sheet_names #Diane
#yesterday = max([pd.to_datetime(elem.split('_')[1]) for elem in vopak_SheetNames]).strftime('%Y%m%d') 
#yesterday = pd.to_datetime(yesterday) - timedelta(days=1)
yesterday = pd.to_datetime(yesterday).strftime('%Y%m%d')
#print(yesterday)
#yesterday = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
sheetName_vopak = 'Vopak_'+ str(yesterday)
vopak_yest = pd.read_excel(path_transporeon + r"Output/ForDiane_Vopak.xlsx", sheet_name=sheetName_vopak, engine='openpyxl', index_col=False)
vopak_yest['Schedule Date'] = pd.to_datetime(vopak_yest['Schedule Date'], format="%Y-%m-%d")
vopak_yest.drop(columns=vopak_yest.columns[0], axis=1, inplace=True)
vopak_yest.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')

#vopak_yest['SAP Order No'] = vopak_yest['SAP Order No'].astype(np.int32)
vopak_yest['SAP Order No'] = pd.to_numeric(vopak_yest['SAP Order No'])
try:
    vopak_yest['Cust Ref'] = vopak_yest['Cust Ref'].astype(np.int32)
except Exception as e:
    print(e)
lst_vopak_yest = vopak_yest['SAP Order No'].tolist()

In [ ]:
yesterday = max([pd.to_datetime(elem.split('_')[1]) for elem in vopak_SheetNames]).strftime('%Y%m%d')
#yesterday = pd.to_datetime(yesterday) - timedelta(days=1)
yesterday #optional

In [30]:
vopak_today = vopak.copy() #Diane
vopak_today['Schedule Date'] = pd.to_datetime(vopak_today['Schedule Date'], format="%Y-%m-%d")
#vopak_today.drop(columns=vopak_today.columns[0], axis=1, inplace=True)
vopak_today.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')

#vopak_today['SAP Order No'] = vopak_today['SAP Order No'].astype(np.int32)
vopak_today['SAP Order No'] = pd.to_numeric(vopak_today['SAP Order No'])
try:
    vopak_today['Cust Ref'] = vopak_today['Cust Ref'].astype(np.int32)
except Exception as e:
    print(e)

lst_vopak_today = vopak_today['SAP Order No'].tolist()

In [31]:
def sameSAP_diffDate(vopak_yest, vopak_today): #Diane
    vopak_yest['SAP_Date'] = list(zip(vopak_yest['SAP Order No'], vopak_yest['Schedule Date'])) #create tuple col from 2 cols
    vopak_today['SAP_Date'] = list(zip(vopak_today['SAP Order No'], vopak_today['Schedule Date']))

    vopak_yest_sapdate = vopak_yest['SAP_Date'].tolist()
    vopak_today_sapdate = vopak_today['SAP_Date'].tolist()

    no_longer_there = set(vopak_yest_sapdate) - set(vopak_today_sapdate)
    new_add = set(vopak_today_sapdate) - set(vopak_yest_sapdate)
    still_there = set(vopak_yest_sapdate) & set(vopak_today_sapdate)

    #next(iter(new_add)) #first elem of set
    #new_add
    vopak_new = vopak_today[vopak_today['SAP_Date'].isin(list(new_add))]
    vopak_new.drop(columns=['SAP_Date'], axis=1, inplace=True)
    return vopak_new

In [32]:
if len(vopak_today) > len(vopak_yest): #Diane
    vopak_new = vopak_today[~vopak_today.isin(vopak_yest)].dropna(how='all')
elif len(vopak_yest) > len(vopak_today):
    #vopak_new = vopak_yest[~vopak_yest.isin(vopak_today)].dropna(how='all')
    s = set(lst_vopak_yest)
    lst_vopak_diff = [x for x in lst_vopak_today if x not in s]
    vopak_new = vopak_today[vopak_today['SAP Order No'].isin(lst_vopak_diff)]
    print(vopak_new)
elif len(vopak_today) == len(vopak_yest):
    s = set(lst_vopak_yest)
    lst_vopak_diff = [x for x in lst_vopak_today if x not in s]
    vopak_new = vopak_today[vopak_today['SAP Order No'].isin(lst_vopak_diff)] #get rows by list
    #print(vopak_new)
else:
    sameSAP_diffDate(vopak_yest, vopak_today)

vopak_email = sameSAP_diffDate(vopak_yest, vopak_today)
vopak_email

C:\Users\john.tan\AppData\Local\Temp\ipykernel_736\3543826728.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vopak_new.drop(columns=['SAP_Date'], axis=1, inplace=True)


,SAP Order No,Schedule Date,Cust Ref,Product name
1,6316639,2024-07-23,1004221280,SOLVESSO 150 AP VL
2,6316640,2024-07-23,1004221270,SOLVESSO 150 AP VL
5,13056952,2024-07-22,1004221139,EXXAL 8 ROP VL
7,13060371,2024-07-23,1004221251,EXXSOL HEXANE AP VL
9,13064192,2024-07-19,1004219206,SOLVESSO 150 AP VL
12,13065027,2024-07-23,1004219883,SOLVESSO 100 AP VL
13,4452584440,2024-07-17,1004218252,NaN


In [ ]:
vopak_email = vopak_email.iloc[:, 1:] #remove 1st col optional

In [33]:
#Vopak email within 10 working days Diane
from datetime import datetime, timedelta 
#acc = pd.read_csv(r"C:/Users/john.tan/Documents/WMS/Accepted_transports.csv")
vopak_email['Schedule Date'] = pd.to_datetime(vopak_email['Schedule Date'])
curr = datetime.now()
end = curr + timedelta(days=14)
start = curr - timedelta(days=1)

vopak_1 = vopak_email[vopak_email['Schedule Date'].between(start, end)]
lst_vopak = vopak_1['Cust Ref'].tolist()
if len(vopak_1) != 0:
    print('\033[1m'+ '\033[32m'+ 'Vopak Email next 10 business days')
    print(vopak_1)
else:
    print('No Vopak transports in next 10 business days')

send_vopak = vopak_1.copy() #make copy of vopak_1 df

Vopak Email next 10 business days
    SAP Order No Schedule Date    Cust Ref        Product name
5       13056952    2024-07-22  1004221139      EXXAL 8 ROP VL
9       13064192    2024-07-19  1004219206  SOLVESSO 150 AP VL
13    4452584440    2024-07-17  1004218252                 NaN


C:\Users\john.tan\AppData\Local\Temp\ipykernel_736\2835122330.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vopak_email['Schedule Date'] = pd.to_datetime(vopak_email['Schedule Date'])


In [34]:
def copyCertainFiles(source_folder, dest_folder, string_to_match, file_type='.pdf'):
    # Check all files in source_folder Diane
    for filename in os.listdir(source_folder):
        # Move the file if the filename contains the string to match
        if file_type == file_type:
            if string_to_match in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

        # Check if the keyword and the file type both match
        elif isinstance(file_type, str):
            if string_to_match in filename and file_type in filename:
                shutil.move(os.path.join(source_folder, filename), dest_folder)

source_folder = path_transporeon + r'Chawan_Vopak/'
dest_folder = path_transporeon + r'Send_Vopak/'
file_type = '.pdf'

for cust_ref in lst_vopak:
    string_to_match = str(cust_ref)
    print(string_to_match)
    try:
        copyCertainFiles(source_folder, dest_folder, string_to_match, file_type)
    except Exception as e:
        print(e)
#close all pdf files
#https://stackoverflow.com/questions/61620485/python-if-filename-in-specified-path-contains-string-then-move-to-folder
#move pdf within 10 working days

1004221139
1004219206
1004218252


In [35]:
#outlook to VOPAK Diane
from importlib import reload #send outlook multiple attachments to multiple recipients 28/11/2022 great!!
import win32com.client as win32
import pythoncom
import sys
import pdfplumber

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def sendmail(receiver, attachment, subject, cc_lst):
    receiver = receiver
    attachment = attachment
    sub = subject
    body = """Dear VOPAK
    <br><br>Kindly release the booking in the system for the orders.
    <br>Customer Name: ExxonMobil Chemical Asia Pacific
    <br>Transporter Name: SH Cogent Logistics Pte Ltd<br><br>""" + vopak_1.to_html() +'<br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')
    mail.CC = ";".join(cc_lst)

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()
    
path_dir = path_transporeon + "Chawan_Vopak/" #copy Vopak pdf to Vopak folder (outlook)
dst = path_transporeon + "Send_Vopak/"
for file in glob.glob(path_dir+"*.pdf"):
    #file = str(file)
    path_from = file
    pdf = pdfplumber.open(file)
    page = pdf.pages[0]
    text = page.extract_text()
    pdf.close()
    try:
        ori = regexp.search(text).group(1).split('SG-',1)[0].split()[1]
        if ori.lower() in ['VOPAK', 'vopak']:
            shutil.copy(file, dst) #if contains 'vopak', copy/paste file
            pdf.close()
    except:
        pass
    
path = path_transporeon + r"Send_Vopak/" # Folder of attachments
addr = pd.read_csv(path_transporeon + "Address.csv") # Email address
cc = pd.read_csv(path_transporeon + "CC.csv") # cc list
cc_lst = cc['CC'].tolist()

for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "VOPAK - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    sendmail(addr["add"][j], dirlist, subjects, cc_lst) # Send emails to multiple recipients with corresponding attachments

John


In [ ]:
def newtrans_mail(receiver_nam, receiver, attachment, subject):
    receiver = receiver
    attachment = attachment
    sub = subject
    receiver_nam = receiver_nam
    body = "Dear "+ receiver_nam + """ user
    <br>(Test email) New transport orders from Transporeon.
    <br><br>""" + df_newtrans.to_html() +"<br><br>SAP Order Number<br>" + str(uploaded_trans) + '<br><br>Download latest pdf from Transporeon<br>' + df_trans.to_html() +'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    for i in range(len(attachment)):
        mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = path_transporeon + "Output/" # Folder of comp.xlsx
addr = pd.read_csv(path_transporeon + "Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "New - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    newtrans_mail(addr["Code"][j], addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

In [ ]:
regexp = re.compile("Destination(.*)$", re.MULTILINE) #TRANSPOREON upload to TMS works 20/1/2023!
regexp_poly = re.compile("PENJURU ROAD(.*)$", re.MULTILINE)

tms_lst = []
tms = pd.DataFrame(columns=['Delivery Type No', 'MV2', 'MV3'])

lst_cust = df_trans['LOADING_PLANT_MV3'].tolist()

#conditions=[cust=='EXXONMOBIL CHEMICAL COMPANY', cust=='MITSUI ELASTOMERS SINGAPORE PTE LTD', cust=='SUMITOMO CHEMICAL PTE LTD',
            #cust=='A.I.M. CHEMICAL INDUSTRIES PTE LTD', cust=='CLP INTERNATIONAL PTE LTD', cust=='CHAMPIONX SG SERVICE PTE LTD',
            #cust=='SINGAPORE ADHESION PLANT', cust=='BASF SOUTH EAST ASIA PTE LTD', cust=='ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            #cust=='MEGACHEM MANUFACTURING PTE LTD', cust=='MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', cust=='SINGAPORE BUTYL PLANT',
            #cust=='BERTSCHI', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG']

#values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA']
#values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1]

for cust in lst_cust:
    cust = cust.strip()
    conditions=[cust=='EXXONMOBIL CHEMICAL COMPANY', cust=='MITSUI ELASTOMERS SINGAPORE PTE LTD', cust=='SUMITOMO CHEMICAL PTE LTD',
            cust=='A.I.M. CHEMICAL INDUSTRIES PTE LTD', cust=='CLP INTERNATIONAL PTE LTD', cust=='CHAMPIONX SG SERVICE PTE LTD',
            cust=='SINGAPORE ADHESION PLANT', cust=='BASF SOUTH EAST ASIA PTE LTD', cust=='ASAHI KASEI SYNTHETIC RUBBER SINGAPORE PTE LTD',
            cust=='MEGACHEM MANUFACTURING PTE LTD', cust=='MSD INTERNATIONAL GMBH (SINGAPORE BRANCH)', cust=='SINGAPORE BUTYL PLANT',
            cust=='BERTSCHI SINGAPORE PTE. LTD.', cust=='BAKER HUGHES', cust=='HP SINGAPORE', cust=='TOTALENERGIES', cust=='SUMITOMO SEIKA-17 SAKRA ROAD PULAU SAKRA ATTN: JOANNE ONG',
            cust=='THE POLYOLEFIN - PULAU AYER MERBAU']

    values=['EXX006', 'MITSUI', 'SUMITOMO CHEMICAL', 'EXX005', 'CLP001', 'CX', 'SAP', 'BASF', 'AKSS', 'MEGA', 'MSD', 'SBP', 'BERTSCHI', '627454', 'EXX002', 'TOTAL', 'SEIKA', 'TPCPAM2']
    values_1 = [1, 4, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4]
    
    MV2 = np.select(conditions, values)
    deli_type = np.select(conditions, values_1)
    print('customer is: '+ str(cust) + ' MV2: '+ str(MV2) + ' Delivery_Type: '+str(deli_type))
    print('------------------')

    #if str(cust).strip() == 'THE POLYOLEFIN - PULAU AYER MERBAU':
            #dest = re.search(regexp_poly, text).group(0)
            #if str(dest) =='PENJURU ROAD 59 STREET 2 GATE NO 5 401' or str(dest) == 'PENJURU ROAD 59 STREET 2 GATE NO 23 401':
                #MV2 = 'TPCPAM2'
                #deli_type = 4
                #print('customer is '+ str(cust) + ' '+ str(MV2) + ' ' + str(deli_type))
            #else:
                #MV2 = 'TPCPAM1'
                #deli_type = 4
                #print('customer is '+ str(dest) + str(MV2) + ' '+ str(deli_type))
    tms_lst.extend([[str(deli_type), str(MV2), cust]])
tms = pd.DataFrame(tms_lst, columns=['Delivery_Type', 'MV2', 'LOADING_PLANT_MV3']) #list of lists to df

In [ ]:
tms

In [ ]:
#tms_upload = pd.concat([tms, df_trans], axis=1)
tms_upload = pd.merge(tms, df_trans, on='LOADING_PLANT_MV3', how='inner')
tms_upload.drop_duplicates(keep='first', inplace=True)
tms_upload.index = np.arange(1, len(tms_upload)+1)
tms_upload #template for TMS upload

In [ ]:
import xlsxwriter #write df_comp

writer = pd.ExcelWriter(path_transporeon + "Output/comp.xlsx", engine='xlsxwriter')
sheetComp = 'Comp_' + datetime.now().strftime("%Y%m%d_%H%M")
df_comp.to_excel(writer, sheet_name=sheetComp) #write to excel with color
worksheet1 = writer.sheets[sheetComp]
worksheet1.set_tab_color('blue')
writer.save()
writer.close()

In [ ]:
def dataframe_difference(df1, df2, which):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    #diff_df.to_csv('data/diff.csv')
    return diff_df

temp_upload_col = ['Cust Ref', 'Transport No', 'SAP Order No']
temp_upload_1[temp_upload_col] = temp_upload_1[temp_upload_col].astype('int64')
new_upload = dataframe_difference(temp_upload_1, master, 'left_only')
new_upload = new_upload.rename(columns={'_merge': 'New_Download'})
new_upload
#temp_upload_2 = temp_upload_1.groupby('LOADING PLANT').apply(lambda x: x.sort_values('SAP Order No'))

In [ ]:
#optional
tp = temp_upload_1.groupby('LOADING PLANT')    
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df

ms = master.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

def repeatRows(d, n):
    return pd.concat([d]*n)

for i in range(min(len(lst_tp), len(lst_ms))):
        
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    if n!= 0:
        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
            #print(len(lst_tp[i]))
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])
            #print(len(lst_ms[i]))
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        print(lst_ms[i].compare(lst_tp[i], keep_shape=True, keep_equal=True)) #compare side by side
        
#https://stackoverflow.com/questions/23691133/split-pandas-dataframe-based-on-groupby

In [ ]:
#alternative comparison optional
tp6 = temp_upload_1.copy()
ms6 = master.copy()
tp6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)
ms6.drop(columns={'Cust Ref', 'Transport No', 'Coll Date', 'DEPOT', 'PRODUCT'}, inplace=True)

diffs = set(ms6.iloc[:, :3].apply(tuple, 1)) \
- set(tp6.apply(tuple, 1))
lst_diffs = list(diffs)

In [ ]:
#optional
lst_tp[11].index = np.arange(1, len(lst_tp[11])+1)
lst_ms[11].index = np.arange(1, len(lst_ms[11])+1)
lst_ms[11].compare(lst_tp[11], keep_shape=True, keep_equal=True)

In [ ]:
# optional (development) comp in color to excel works 17/1/2023!
temp_upload_6 = temp_upload_1.copy()
master_6 = master.copy()
tp = temp_upload_6.groupby('LOADING PLANT')   
lst_tp = [tp.get_group(x) for x in tp.groups] #groupby and split df
ms = master_6.groupby('LOADING PLANT')
lst_ms = [ms.get_group(x) for x in ms.groups]

for i in range(min(len(lst_tp), len(lst_ms))):
    n = abs(len(lst_ms[i]) - len(lst_tp[i])) #ensure both df same len
    
    if n!= 0:

        if len(lst_ms[i]) > len(lst_tp[i]):
            lst_tp[i] = pd.concat([lst_tp[i], repeatRows(lst_tp[i][-1:], n)])
        else:
            lst_ms[i] = pd.concat([lst_ms[i], repeatRows(lst_ms[i][-1:], n)])           
          
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)      
           
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        #df_comp
        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/Comp_color/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')
               

    if n ==0:    
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
        df_all = pd.concat([lst_tp[i], lst_ms[i]], axis='columns', keys=['First', 'Second'])
        df_comp = df_all.swaplevel(axis='columns')[lst_tp[i].columns[1:]]

        lst_ms[i] = lst_ms[i].dropna(subset=['Transport No','Coll Date'], how='all')
        lst_tp[i] = lst_tp[i].dropna(subset=['Transport No','Coll Date'], how='all')
            
        #lst_tp[i] = lst_tp[i].iloc[:, :-1]
        lst_tp[i].index = np.arange(1, len(lst_tp[i])+1)
        lst_ms[i].index = np.arange(1, len(lst_ms[i])+1)
                
        df_comp = df_comp[(lst_tp[i] != lst_ms[i]).any(1)].style.apply(highlight_diff, axis=None)
        #df_comp[(temp_upload_1 != master).any(1)].style.apply(highlight_diff, axis=None)

        with pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/Transporeon/Output/comp.xlsx", mode="a", engine="openpyxl") as writer:
            df_comp.to_excel(writer, sheet_name='comp_'+str(i)+'_'+datetime.now().strftime("%Y-%m-%d"), engine='openpyxl')

In [ ]:
#compare Upcoming with master. Works 02/02/2023!
upc = pd.read_csv(path_transporeon + "Upcoming.csv", usecols=['Order No'])
upc.rename(columns={'Order No': 'SAP Order No'}, inplace=True) #'Deliv Date': 'DELIVERY'
#upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
upc.index = np.arange(1, len(upc)+1)
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(path_transporeon + "EM master file.csv", usecols=['EM SO# Ref'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
master_upc.index = np.arange(1, len(master_upc)+1)

sap_master_upc = master_upc['SAP Order No'].tolist()

master_upc_1 = master_upc.copy()
master_upc_1['SAP Order No'] = master_upc_1['SAP Order No'].astype(np.int64) #col to int
upOnly = dataframe_difference(upc, master_upc_1, 'left_only')

upOnly.rename(columns={"_merge": "New_Transport"}, inplace=True)
upOnly['New_Transport'] = upOnly['New_Transport'].replace('left_only', 'new upcoming')
upOnly.index = np.arange(1, len(upOnly)+1)
print('\033[1m'+ '\033[34m'+ magenta(('Transports from Upcoming Shipments Report not in MasterFile'), ['bold', 'underlined'])) #'\033[94m'
upOnly

In [ ]:
upc['SAP Order No'] = upc['SAP Order No'].astype(np.int64)
master_upc_1['SAP Order No'] = master_upc_1['SAP Order No'].astype(np.int64)
arr_upc = np.array(lst_upc)
arr_master_upc = np.array(sap_master_upc)
dif1 = np.setdiff1d(arr_upc, arr_master_upc)
dif2 = np.setdiff1d(arr_master_upc, arr_upc)

lst_upOnly = np.concatenate((dif1, dif2))
print('\033[1m'+ '\033[34m'+ 'SAP Numbers from Upcoming Shipments Report not in MasterFile')
lst_upOnly

In [ ]:
upc_only = pd.read_csv(path_transporeon + "Upcoming.csv")
upc_only['Order No'] = upc_only['Order No'].astype('str')
upc_only = upc_only[upc_only['Order No'].isin(lst_upOnly)]
print('\033[1m'+ '\033[34m'+ 'Upcoming Shipments Report not in MasterFile')
upc_only

In [ ]:
#compare Upcoming with master (development)
upc = pd.read_csv(path_transporeon + "Upcoming.csv", usecols=['Order No', 'Deliv Date', 'Customer Name'])
upc.rename(columns={'Order No': 'SAP Order No', 'Customer Name': 'LOADING', 'Deliv Date': 'DELIVERY'}, inplace=True)
upc['DELIVERY'] = pd.to_datetime(upc['DELIVERY'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
lst_upc = upc['SAP Order No'].tolist()

new_upc = [] #convert list of int to str
for x in lst_upc:
    str_x = "% s" % x
    new_upc.append(str_x)

master_upc = pd.read_csv(path_transporeon + "EM master file.csv", usecols=['EM SO# Ref', 'DELIVERY', 'LOADING PLANT'])
master_upc.rename(columns={'EM SO# Ref': 'SAP Order No', 'LOADING PLANT': 'LOADING'}, inplace=True)
master_upc = master_upc[master_upc['SAP Order No'].isin(new_upc)]
master_upc.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')

sap_master_upc = master_upc['SAP Order No'].tolist()
#print(sap_master_upc) #list of str
#print(lst_upc) #list of int

upc['SAP Order No'] = upc['SAP Order No'].astype(str) #convert col to str
upc_notin_master = upc[~upc['SAP Order No'].isin(sap_master_upc)] #select Upcoming not in Master
upc_notin_master.index = np.arange(1, len(upc_notin_master)+1)
#print('Upcoming transports not in Master file')
#print(upc_notin_master)

idx1 = set(upc.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
idx2 = set(upc_notin_master.set_index(['SAP Order No', 'LOADING', 'DELIVERY']).index)
upc_in_master = pd.DataFrame(list(idx1 - idx2), columns=upc.columns) #select Upcoming in Master

upc_in_master.sort_values(['SAP Order No'], ascending=True, inplace=True, na_position='first')
upc_in_master.index = np.arange(1, len(upc_in_master)+1)
print('Upcoming transports in Master file')
print(upc_in_master)

master_upc.index = np.arange(1, len(master_upc)+1)
print('Master file')
print(master_upc)

#diffs = set(master_upc.iloc[:, :3].apply(tuple, 1)) \
                       #-  set(upc.apply(tuple, 1))
#print('Difference in master ')
#print(diffs)
diffs_2 = set(upc.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master_upc.apply(tuple, 1))
#print('New entry in Upcoming transports')
DF_diffs_2 = pd.DataFrame(data=list(diffs_2), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
#DF_diffs_2
#https://stackoverflow.com/questions/46635439/compare-rows-of-two-data-frames-with-unequal-lengths
#https://stackoverflow.com/questions/37313691/how-to-remove-a-pandas-dataframe-from-another-dataframe

In [ ]:
from importlib import reload #send outlook multiple recipients 18/1/2023 great!!
import win32com.client as win32
import pythoncom

reload(sys)
pythoncom.CoInitialize()
outlook = win32.Dispatch('outlook.application')

def upctrans_mail(receiver, attachment, subject):
    receiver = receiver
    #attachment = attachment
    sub = subject
    body = """Dear User
    <br>(Test email) EM MasterFile.
    <br><br>""" + master_upc.to_html() +"<br><br>Upcoming Shipments not in MasterFile<br>" + str(lst_upOnly)+'<br>'+ upc_only.to_html()+'<br><br>This message is computer generated. '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    mail = outlook.CreateItem(0)
    mail.To = receiver
    mail.subject = sub.encode('utf-8').decode('utf-8')
    mail.HTMLBody = body.encode('utf-8').decode('utf-8')

    #for i in range(len(attachment)):
        #mail.Attachments.Add(attachment[i]) # Upload multiple attachments
    mail.Send()

##outlook to user download latest pdf Transporeon, comp.xlsx
path = path_transporeon + "Upcoming/" # Folder of comp.xlsx
addr = pd.read_csv(path_transporeon + "Address.csv") # Email address
    
for j in range(len(addr.Code)):
    dirlist = []
# Address spreadsheet includes “Code” which is the recipient’s name, and “add”, the email address
    sub_code = addr.Code[j]
    #subjects = sub_code + " - transports"
    subjects = "Upcoming - transports"
    for dirpath,dirname,filename in os.walk(path):
        print(addr.Code[j])
        for i in filename:
            #if i[0:9] == 'Transport':
                dirlist.append(os.path.join(dirpath,i))
           
    upctrans_mail(addr["add"][j], dirlist, subjects) # Send emails to multiple recipients with corresponding attachments

In [ ]:
temp_upload_diff = temp_upload_1[['SAP Order No', 'LOADING PLANT', 'DELIVERY']] #optional
temp_upload_diff.rename(columns={'LOADING PLANT': 'LOADING'}, inplace=True)
diffs_3 = set(temp_upload_diff.iloc[:, :3].apply(tuple, 1)) \
                       -  set(master.apply(tuple, 1))
print('New entry in Uploaded transports')
DF_diffs_3 = pd.DataFrame(data=list(diffs_3), columns=['SAP Order No', 'LOADING', 'DELIVERY'])
DF_diffs_3 #Upcoming not in master

In [ ]:
#df3 = df2.reindex_like(df1) #optional
#df1[(df3['a'].isin(df1['a'])) & (df1['b'] <= df3['b'])]
comp_6 = temp_upload_1.reindex_like(master)
master[(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['PRODUCT'] <= comp_6['PRODUCT'])]
#master[np.logical_not(comp_6['Cust Ref'].isin(master['Cust Ref'])) & (master['DEPOT'] <= comp_6['DEPOT'])]

In [ ]:
#writer1 = pd.ExcelWriter(r"C:/Users/john.tan/OneDrive - Cogent Holdings Pte. Ltd/Documents/WMS/Transporeon_Test/temp_upload.xlsx", engine='xlsxwriter')
#temp_upload_1.to_excel(writer1, sheet_name='temp_upload') #write to excel with color
#worksheet1 = writer1.sheets['temp_upload']
#worksheet1.set_tab_color('blue')
#writer1.save()
#writer1.close()

with pd.ExcelWriter(path_transporeon + "Output/NewTransports_ChawanVopak.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer3:
    sheetName = 'NewTransports_' + datetime.now().strftime("%Y%m%d %H%M")
    temp_upload_1.to_excel(writer3, sheet_name=sheetName, engine='openpyxl')
    worksheet3 = writer3.sheets[sheetName]
    writer3.save()

with pd.ExcelWriter(path_transporeon + "Output/NewTransports_50.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer4:
    sheetName = 'Transporeon50_' + datetime.now().strftime("%Y%m%d %H%M")
    df_trans.to_excel(writer4, sheet_name=sheetName, engine='openpyxl')
    worksheet4 = writer4.sheets[sheetName]
    writer4.save()
  
with pd.ExcelWriter(path_wms + "NewTransports.xlsx", mode='a', if_sheet_exists='replace', engine='openpyxl') as writer5:
    sheetName = 'NewTransports_' + datetime.now().strftime("%Y%m%d %H%M")
    df_newtrans.to_excel(writer5, sheet_name=sheetName, engine='openpyxl')
    worksheet5 = writer5.sheets[sheetName]
    writer5.save()
print('\033[1m'+ '\033[32m' + 'Process completed')

In [ ]:
path_transporeon = r"C:/Users/john.tan/Documents/Transporeon/"
path_wms = r"C:/Users/john.tan/Documents/WMS/"

In [ ]:
#development kv_pairs to df #optional
for kv_pair in result.key_value_pairs:
            if kv_pair.key and kv_pair.value:
                lst_trans.extend([[kv_pair.key.content, kv_pair.value.content]])
                #print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
            else:
                print("Key '{}': Value:".format(kv_pair.key.content))
                print('----------------------')
  
            items = result.key_value_pairs    
            #i += 1
#lst_trans

#field_lst = ["Cust Ref", "Order Remarks", "Transport No", "Coll Date", "DEPOT_MV2", "LOADING_PLANT_MV3", "DELIVERY", "SAP Order No", "MV2"]
field_lst = ['Shipping Number (BOL)', 'Comment', 'Transport No.', 'Deadline', 'Origin', 'Destination',  'Deadline', 'SAP Order No.', 'Article description']#MV2
field_lst2 = ['Comment', 'Transport No.', 'Deadline', 'Origin', 'Destination',  'Deadline', 'SAP Order No.', 'Article description']#MV2
qq = pd.DataFrame(lst_trans, columns=['SN', 'Va'])
qqq = qq.set_index('SN').T.reset_index(drop=True)
try:
    q = qqq[field_lst]
except:
    pass
    q = qqq[field_lst2]
q = q.loc[:, ~q.columns.duplicated()].copy() #remove dup cols
q.rename(columns={'Comment': 'Order Remarks', 'Transport No.': 'Transport No', 'Deadline': 'Coll Date', 'Origin': 'DEPOT_MV2', 'Destination': 'LOADING_PLANT_MV3', 'SAP Order No.': 'SAP Order No', 'Article description': 'PRODUCT'}, inplace=True)

q['TIME SLOT'] = [x.split(' ')[1] for x in q['Coll Date']]
q['Coll Date'] = [x.split(' ')[0].replace(' ', '') for x in q['Coll Date']]
q['DELIVERY'] = q['Coll Date']
#q.drop(columns=q.columns[0], axis=1, inplace=True)

col_reord = ['Order Remarks', 'Transport No', 'Coll Date', 'DEPOT_MV2', 'LOADING_PLANT_MV3', 'DELIVERY', 'SAP Order No', 'PRODUCT', 'TIME SLOT']
new_cols = col_reord + (q.columns.drop(col_reord).tolist())
q = q[new_cols]
q

In [ ]:
for kv_pair in result.key_value_pairs:
            if kv_pair.key and kv_pair.value:
                lst_trans.extend([[kv_pair.key.content, kv_pair.value.content]])
                #print("Key '{}': Value: '{}'".format(kv_pair.key.content, kv_pair.value.content))
lst_trans          